In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 경로 설정 및 코호트 불러오기 (cohort_ver48)

In [ ]:
import pandas as pd
import numpy as np
import os

# -----------------------------------------------------------
# 1. 경로 설정 (Path Configuration)
# -----------------------------------------------------------
# 기본 경로
base_path = '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스'

# ED 데이터셋 폴더 경로 (mimid -> mimic으로 수정했습니다. 실제 폴더명 확인 필요)
ed_base_path = os.path.join(base_path, 'mimic-iv-ed-2.2')
edstays_csv_path = os.path.join(ed_base_path, 'edstays.csv')

# 코호트 파일 경로 (base_path 내에 있다고 가정)
cohort_csv_path = os.path.join(base_path, 'cohort_ver48_remove_trash_col.csv')

# -----------------------------------------------------------
# 2. 데이터 로드 (Data Loading)
# -----------------------------------------------------------
try:
    print(f"Loading ED Stays from: {edstays_csv_path}")
    df_edstays = pd.read_csv(edstays_csv_path, dtype={'subject_id': int, 'hadm_id': 'Int64', 'stay_id': int})
    print(f" -> edstays shape: {df_edstays.shape}")

    print(f"Loading Cohort from: {cohort_csv_path}")
    # [수정 완료] os.path.join 함수가 아닌, 완성된 문자열 경로(cohort_csv_path)를 넣었습니다.
    df_cohort = pd.read_csv(cohort_csv_path)
    print(f" -> cohort shape: {df_cohort.shape}")

except FileNotFoundError as e:
    print(f"\n[오류] 파일을 찾을 수 없습니다. 경로를 확인해주세요.\nError: {e}")
except Exception as e:
    print(f"\n[오류] 데이터 로드 중 문제가 발생했습니다.\nError: {e}")

# -----------------------------------------------------------
# 3. 날짜 컬럼 변환 (Datetime Conversion)
# -----------------------------------------------------------
if 'df_cohort' in locals():
    time_cols = [
        'ed_intime', 'ed_outtime', 'first_ecg_charttime',
        'first_troponin_charttime', 'first_antithrombotic_charttime',
        'first_stemi_ecg_time', 'first_troponin_positive_charttime',
        'pci_starttime', 'pci_endtime', 'dischtime'
    ]

    print("\n변환 중: 날짜 컬럼 datetime 변환...")
    for col in time_cols:
        if col in df_cohort.columns:
            df_cohort[col] = pd.to_datetime(df_cohort[col], errors='coerce')

    print("날짜 변환 완료.")

# [1] event log 형태로 테이블 변환

In [ ]:
# 2. 정적 데이터(arrival_transport) 인코딩 및 병합
# 만약 코호트에 이미 arrival_transport가 있다면 그대로 사용, 없다면 edstays와 조인
if 'arrival_transport' not in df_cohort.columns and 'df_edstays' in locals():
    df_cohort = df_cohort.merge(df_edstays[['hadm_id', 'arrival_transport']], on='hadm_id', how='left')

def encode_transport(x):
    if pd.isna(x): return 0
    x = str(x).upper()
    if 'WALK IN' in x: return 1
    elif 'AMBULANCE' in x: return 2
    elif 'HELICOPTER' in x: return 3
    else: return 0

# 인코딩 적용
if 'arrival_transport' in df_cohort.columns:
    df_cohort['arrival_transport_code'] = df_cohort['arrival_transport'].apply(encode_transport)

# 3. 정적 코호트 -> 동적 이벤트 로그로 변환 (Melting)
# 주요 타임스탬프를 이벤트로 정의하여 행(Row)을 늘립니다.
event_mapping = {
    'ed_intime': 'ED_ARRIVAL',
    'first_ecg_charttime': 'ECG_TAKEN',
    'first_troponin_charttime': 'TROP_TAKEN',
    'first_antithrombotic_charttime': 'ANTI_TAKEN',
    'first_stemi_ecg_time': 'STEMI_FLAG',
    'pci_starttime': 'PCI_START',
    'pci_endtime': 'PCI_END',
    'dischtime': 'DISCHARGE'
}

events_list = []
for col, event_name in event_mapping.items():
    if col in df_cohort.columns:
        temp = df_cohort[['subject_id', 'hadm_id', col]].dropna()
        temp.columns = ['subject_id', 'hadm_id', 'charttime']
        temp['event_name'] = event_name
        events_list.append(temp)

# 모든 이벤트를 합치고 정렬
df_events = pd.concat(events_list).sort_values(by=['subject_id', 'hadm_id', 'charttime'])

# 원본 코호트의 기준 시간(ed_intime 등)을 다시 병합 (시간 차이 계산용)
df_events = df_events.merge(df_cohort[['hadm_id', 'ed_intime', 'first_stemi_ecg_time', 'first_troponin_positive_charttime']], on='hadm_id', how='left')

df_events

,subject_id,hadm_id,charttime,event_name,ed_intime,first_stemi_ecg_time,first_troponin_positive_charttime
0,10000764,27897940,2132-10-14 19:31:00,ED_ARRIVAL,2132-10-14 19:31:00,2132-10-14 19:40:00,2132-10-15 07:45:00
1,10000764,27897940,2132-10-14 19:40:00,ECG_TAKEN,2132-10-14 19:31:00,2132-10-14 19:40:00,2132-10-15 07:45:00
2,10000764,27897940,2132-10-14 19:40:00,STEMI_FLAG,2132-10-14 19:31:00,2132-10-14 19:40:00,2132-10-15 07:45:00
3,10000764,27897940,2132-10-15 07:45:00,TROP_TAKEN,2132-10-14 19:31:00,2132-10-14 19:40:00,2132-10-15 07:45:00
4,10000764,27897940,2132-10-15 20:00:00,ANTI_TAKEN,2132-10-14 19:31:00,2132-10-14 19:40:00,2132-10-15 07:45:00
...,...,...,...,...,...,...,...
11170,19996783,21880161,2188-03-05 15:18:00,ECG_TAKEN,2188-05-09 05:02:00,NaT,2188-05-09 21:56:00
11171,19996783,21880161,2188-05-09 05:02:00,ED_ARRIVAL,2188-05-09 05:02:00,NaT,2188-05-09 21:56:00
11172,19996783,21880161,2188-05-09 21:56:00,TROP_TAKEN,2188-05-09 05:02:00,NaT,2188-05-09 21:56:00
11173,19996783,21880161,2188-05-09 23:00:00,ANTI_TAKEN,2188-05-09 05:02:00,NaT,2188-05-09 21:56:00


# [2] 동적 피처 엔지니어링

In [ ]:
# -----------------------------------------------------------
# 4. 동적 피처 엔지니어링 (Dynamic Feature Engineering)
# -----------------------------------------------------------

# 4-1. Time Features
df_events['current_time'] = df_events['charttime']
# time_since_ed (분 단위)
df_events['time_since_ed'] = (df_events['current_time'] - df_events['ed_intime']).dt.total_seconds() / 60
# time_since_last (직전 이벤트로부터 경과 시간)
df_events['prev_event_time'] = df_events.groupby('hadm_id')['charttime'].shift(1)
df_events['time_since_last'] = (df_events['current_time'] - df_events['prev_event_time']).dt.total_seconds() / 60
df_events['time_since_last'] = df_events['time_since_last'].fillna(0) # 첫 이벤트는 0

# time_since_start_min (첫 이벤트로부터 경과 시간)
df_events['first_event_time'] = df_events.groupby('hadm_id')['charttime'].transform('min')
df_events['time_since_start_min'] = (df_events['current_time'] - df_events['first_event_time']).dt.total_seconds() / 60

# is_night (22시 ~ 07시)
df_events['hour'] = df_events['current_time'].dt.hour
df_events['is_night'] = df_events['hour'].apply(lambda h: 1 if (h >= 22 or h < 7) else 0)

# 4-2. Sequence Features
# prefix_len
df_events['prefix_len'] = df_events.groupby('hadm_id').cumcount() + 1

# prefix_events_str (수정됨)
# expanding() 대신 문자열 덧셈과 cumsum()을 사용하여 누적 문자열 생성
# 1. 각 이벤트 이름 뒤에 구분자 '>'를 붙임
# 2. 그룹별로 누적 합(문자열 연결)을 구함
# 3. 마지막에 붙은 불필요한 '>'를 제거
df_events['prefix_events_str'] = df_events.groupby('hadm_id')['event_name'].transform(
    lambda x: (x.astype(str) + '>').cumsum().str[:-1]
)

# 4-3. Count & State Features
# (주의: 제공된 코호트 파일에는 개별 트로포닌 수치가 없고 '총 횟수'만 있으므로, 여기서는 이벤트 발생 여부로 카운트합니다)
df_events['is_ecg'] = (df_events['event_name'] == 'ECG_TAKEN').astype(int)
df_events['cum_ecg_cnt'] = df_events.groupby('hadm_id')['is_ecg'].cumsum()

df_events['is_trop'] = (df_events['event_name'] == 'TROP_TAKEN').astype(int)
df_events['cum_trop_cnt'] = df_events.groupby('hadm_id')['is_trop'].cumsum()

# stemi_flag (확진 시점 이후면 1)
df_events['stemi_flag'] = 0
mask_stemi = (df_events['first_stemi_ecg_time'].notna()) & (df_events['current_time'] >= df_events['first_stemi_ecg_time'])
df_events.loc[mask_stemi, 'stemi_flag'] = 1

# trop_pos_flag (양성 판정 시점 이후면 1)
df_events['trop_pos_flag'] = 0
mask_trop_pos = (df_events['first_troponin_positive_charttime'].notna()) & (df_events['current_time'] >= df_events['first_troponin_positive_charttime'])
df_events.loc[mask_trop_pos, 'trop_pos_flag'] = 1

# pci_status
# 0:안함, 1:진행중, 2:종료
# 간단 로직: PCI_START 이벤트 시점=1, PCI_END 이후=2
df_events['pci_status'] = 0
df_events.loc[df_events['event_name'] == 'PCI_START', 'pci_status'] = 1
df_events.loc[df_events['event_name'] == 'PCI_END', 'pci_status'] = 2
# (실제로는 상태 유지 로직(forward fill)이 필요할 수 있으나, 이벤트 시점 기준으로는 위와 같이 태깅)

# 4-4. Pathway Stage Mapping
stage_map = {
    'ED_ARRIVAL': 0,
    'ECG_TAKEN': 1,
    'TROP_TAKEN': 1,
    'STEMI_FLAG': 2,
    'ANTI_TAKEN': 3,
    'PCI_START': 4,
    'PCI_END': 5,
    'DISCHARGE': 5
}
df_events['pathway_stage'] = df_events['event_name'].map(stage_map).fillna(0)

# 4-5. ID Mapping
event_id_map = {name: i for i, name in enumerate(stage_map.keys())}
df_events['current_event_id'] = df_events['event_name'].map(event_id_map)

# 4-6. Lab Values (last_trop, run_max_trop, trop_trend)
# 주의: 현재 코호트 파일에는 구체적인 Lab Value(검사 수치)가 없습니다.
# 실제 Lab 데이터가 있다면 merge 후 아래와 같이 처리합니다.
# df_events['last_trop'] = df_events.groupby('hadm_id')['trop_value'].ffill().fillna(0)
# df_events['run_max_trop'] = df_events.groupby('hadm_id')['trop_value'].expanding().max().fillna(0)
# df_events['trop_trend'] = df_events['last_trop'] - df_events.groupby('hadm_id')['last_trop'].shift(1).fillna(0)

# 최종 결과 확인
cols_order = ['subject_id', 'hadm_id', 'prefix_len', 'event_name', 'charttime', 'time_since_ed', 'stemi_flag', 'pathway_stage']
print(df_events[cols_order].head(10))

   subject_id   hadm_id  prefix_len  event_name           charttime  \
0    10000764  27897940           1  ED_ARRIVAL 2132-10-14 19:31:00   
1    10000764  27897940           2   ECG_TAKEN 2132-10-14 19:40:00   
2    10000764  27897940           3  STEMI_FLAG 2132-10-14 19:40:00   
3    10000764  27897940           4  TROP_TAKEN 2132-10-15 07:45:00   
4    10000764  27897940           5  ANTI_TAKEN 2132-10-15 20:00:00   
5    10000764  27897940           6   DISCHARGE 2132-10-19 16:30:00   
6    10010058  26359957           1   ECG_TAKEN 2119-04-03 09:52:00   
7    10010058  26359957           2  STEMI_FLAG 2143-10-21 10:40:00   
8    10010058  26359957           3  ED_ARRIVAL 2147-11-18 00:50:00   
9    10010058  26359957           4  ANTI_TAKEN 2147-11-18 05:00:00   

   time_since_ed  stemi_flag  pathway_stage  
0            0.0           0              0  
1            9.0           1              1  
2            9.0           1              2  
3          734.0           1      

# [3] Lab 수치 피처 불러오기 (hosp-icu DB)

In [ ]:
import sqlite3
import pandas as pd
import numpy as np

# -----------------------------------------------------------
# [설정] DB 경로 및 타겟 설정
# -----------------------------------------------------------
db_path = '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/MIMIC4-hosp-icu.db'

# 트로포닌 Item ID (Troponin I: 51002, Troponin T: 51003)
trop_itemids = (51002, 51003)

# 코호트의 hadm_id 목록 추출 (SQL 필터링용)
# 1. NaN 제거
# 2. 정수형(int)으로 변환 (numpy int64가 아닌 python int)
# 3. 튜플로 변환
target_hadm_ids = tuple(map(int, df_cohort['hadm_id'].dropna().unique()))

# -----------------------------------------------------------
# 1. Lab 데이터 로드 (SQLite Query)
# -----------------------------------------------------------
print("DB에서 Troponin 데이터 조회 중...")

conn = sqlite3.connect(db_path)

# SQL 쿼리 작성 시, IN 절에 들어갈 문자열을 안전하게 생성합니다.
# 튜플을 문자열로 변환 (예: "(123, 456, ...)")
ids_str = str(target_hadm_ids)
itemids_str = str(trop_itemids)

# 쿼리문 (f-string 사용하되, 내용은 순수 숫자 문자열임)
query = f"""
SELECT subject_id, hadm_id, itemid, charttime, valuenum, flag
FROM labevents
WHERE itemid IN {itemids_str}
  AND hadm_id IN {ids_str}
"""

try:
    # 쿼리 실행 및 데이터프레임 로드
    df_trop = pd.read_sql_query(query, conn)
    print(f"Troponin 데이터 로드 완료: {len(df_trop)} 건")
except Exception as e:
    print(f"데이터 로드 중 오류 발생: {e}")
    # 오류 메시지를 자세히 보기 위해 쿼리 앞부분 출력 (디버깅용)
    print(f"Query snippet: {query[:200]} ...")
    df_trop = pd.DataFrame()
finally:
    conn.close()

# -----------------------------------------------------------
# (이후 Lab 피처 엔지니어링 코드는 기존과 동일하게 진행)
# -----------------------------------------------------------
if not df_trop.empty:
    # 날짜 변환 및 정렬
    df_trop['charttime'] = pd.to_datetime(df_trop['charttime'])
    df_trop = df_trop.sort_values(by=['hadm_id', 'charttime'])

    # 결측치 처리 (수치가 없는 경우 0 처리)
    df_trop['valuenum'] = df_trop['valuenum'].fillna(0)

    # [Feature 1] last_trop (현재 측정값)
    df_trop['last_trop'] = df_trop['valuenum']

    # [Feature 2] run_max_trop (현재 시점까지의 최대값 - Running Max)
    # expanding().max()를 사용하여 누적 최대값을 구합니다.
    df_trop['run_max_trop'] = df_trop.groupby('hadm_id')['valuenum'].expanding().max().reset_index(level=0, drop=True)

    # [Feature 3] trop_trend (현재값 - 직전값)
    # shift(1)을 사용하여 직전 검사 결과를 가져옵니다.
    df_trop['prev_trop'] = df_trop.groupby('hadm_id')['valuenum'].shift(1).fillna(0)
    df_trop['trop_trend'] = df_trop['last_trop'] - df_trop['prev_trop']

    # 이벤트 이름 부여 (기존 로그와 합치기 위해)
    df_trop['event_name'] = 'TROP_TAKEN'

    # -----------------------------------------------------------
    # 3. 기존 이벤트 로그와 병합 (Merge Strategy)
    # -----------------------------------------------------------
    # 앞 단계에서 생성한 df_events가 있다고 가정
    if 'df_events' in locals():
        # 3-1. 기존 로그에서 내용 없는 껍데기 'TROP_TAKEN' 제거
        df_events_no_trop = df_events[df_events['event_name'] != 'TROP_TAKEN'].copy()

        # 3-2. Lab 데이터에 필요한 컬럼만 남기기
        lab_cols = ['subject_id', 'hadm_id', 'charttime', 'event_name', 'last_trop', 'run_max_trop', 'trop_trend']
        df_trop_merge = df_trop[lab_cols]

        # 3-3. 병합 및 재정렬
        df_final = pd.concat([df_events_no_trop, df_trop_merge], ignore_index=True)
        df_final = df_final.sort_values(by=['subject_id', 'hadm_id', 'charttime'])

        # 3-4. 결측값 채우기 (Forward Fill)
        # 검사하지 않은 시점(예: ECG)에도 가장 최근의 Lab 수치를 유지
        lab_features = ['last_trop', 'run_max_trop']
        df_final[lab_features] = df_final.groupby('hadm_id')[lab_features].ffill().fillna(0)
        df_final['trop_trend'] = df_final['trop_trend'].fillna(0)

        # 3-5. Time Feature 재계산 (Merge로 인해 깨진 시간 정보 복구)
        df_final['current_time'] = df_final['charttime']

        if 'ed_intime' not in df_final.columns:
             df_final = df_final.merge(df_cohort[['hadm_id', 'ed_intime']], on='hadm_id', how='left')

        df_final['time_since_ed'] = (df_final['current_time'] - df_final['ed_intime']).dt.total_seconds() / 60

        print("최종 피처 테이블 생성 완료 (df_final)")
        print(df_final[['hadm_id', 'event_name', 'run_max_trop']].head())
    else:
        print("경고: df_events 변수가 정의되지 않았습니다. 이전 단계 코드를 먼저 실행하세요.")
else:
    print("경고: 해당 코호트의 Troponin 데이터를 찾을 수 없습니다.")

df_final

DB에서 Troponin 데이터 조회 중...
Troponin 데이터 로드 완료: 4013 건
최종 피처 테이블 생성 완료 (df_final)
        hadm_id  event_name  run_max_trop
0      27897940  ED_ARRIVAL          0.00
1      27897940   ECG_TAKEN          0.00
2      27897940  STEMI_FLAG          0.00
13042  27897940  TROP_TAKEN          0.04
3      27897940  ANTI_TAKEN          0.04


,subject_id,hadm_id,charttime,event_name,ed_intime,first_stemi_ecg_time,first_troponin_positive_charttime,current_time,time_since_ed,prev_event_time,...,is_trop,cum_trop_cnt,stemi_flag,trop_pos_flag,pci_status,pathway_stage,current_event_id,last_trop,run_max_trop,trop_trend
0,10000764,27897940,2132-10-14 19:31:00,ED_ARRIVAL,2132-10-14 19:31:00,2132-10-14 19:40:00,2132-10-15 07:45:00,2132-10-14 19:31:00,0.0,NaT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
1,10000764,27897940,2132-10-14 19:40:00,ECG_TAKEN,2132-10-14 19:31:00,2132-10-14 19:40:00,2132-10-15 07:45:00,2132-10-14 19:40:00,9.0,2132-10-14 19:31:00,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.00,0.00,0.00
2,10000764,27897940,2132-10-14 19:40:00,STEMI_FLAG,2132-10-14 19:31:00,2132-10-14 19:40:00,2132-10-15 07:45:00,2132-10-14 19:40:00,9.0,2132-10-14 19:40:00,...,0.0,0.0,1.0,0.0,0.0,2.0,3.0,0.00,0.00,0.00
13042,10000764,27897940,2132-10-15 07:45:00,TROP_TAKEN,NaT,NaT,NaT,2132-10-15 07:45:00,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.04,0.04,0.04
3,10000764,27897940,2132-10-15 20:00:00,ANTI_TAKEN,2132-10-14 19:31:00,2132-10-14 19:40:00,2132-10-15 07:45:00,2132-10-15 20:00:00,1469.0,2132-10-15 07:45:00,...,0.0,1.0,1.0,1.0,0.0,3.0,4.0,0.04,0.04,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9831,19996783,21880161,2188-05-09 05:02:00,ED_ARRIVAL,2188-05-09 05:02:00,NaT,2188-05-09 21:56:00,2188-05-09 05:02:00,0.0,2188-03-05 15:18:00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
10548,19996783,21880161,2188-05-09 21:56:00,TROP_TAKEN,NaT,NaT,NaT,2188-05-09 21:56:00,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.37,1.37,1.37
9832,19996783,21880161,2188-05-09 23:00:00,ANTI_TAKEN,2188-05-09 05:02:00,NaT,2188-05-09 21:56:00,2188-05-09 23:00:00,1078.0,2188-05-09 21:56:00,...,0.0,1.0,0.0,1.0,0.0,3.0,4.0,1.37,1.37,0.00
10549,19996783,21880161,2188-05-10 03:56:00,TROP_TAKEN,NaT,NaT,NaT,2188-05-10 03:56:00,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.92,1.92,0.55


In [ ]:
# =============================================================================
# [Step 4.5] 병합 후 피처 재계산 (Repair Missing Values)
# =============================================================================
# 설명: Lab 데이터(df_trop)가 병합되면서 비어있는 ed_intime과 동적 피처들을 다시 채웁니다.

# 1. ed_intime 및 주요 시간 정보 채우기
# hadm_id별로 ed_intime이 같은 환자라면 동일하므로 forward fill/backward fill로 채웁니다.
time_fill_cols = ['ed_intime', 'first_stemi_ecg_time', 'first_troponin_positive_charttime']
for col in time_fill_cols:
    if col in df_final.columns:
        df_final[col] = df_final.groupby('hadm_id')[col].transform('first') # 그룹 내 첫 번째 값으로 채움

# 2. Time Features 재계산 (ed_intime이 채워졌으므로 가능)
df_final['current_time'] = df_final['charttime']
df_final['time_since_ed'] = (df_final['current_time'] - df_final['ed_intime']).dt.total_seconds() / 60

# prev_event_time & time_since_last 재계산 (새로 들어온 Lab 이벤트 순서 반영)
df_final = df_final.sort_values(by=['subject_id', 'hadm_id', 'charttime']) # 시간순 정렬 필수
df_final['prev_event_time'] = df_final.groupby('hadm_id')['charttime'].shift(1)
df_final['time_since_last'] = (df_final['current_time'] - df_final['prev_event_time']).dt.total_seconds() / 60
df_final['time_since_last'] = df_final['time_since_last'].fillna(0)

# is_night 재계산
df_final['hour'] = df_final['current_time'].dt.hour
df_final['is_night'] = df_final['hour'].apply(lambda h: 1 if (h >= 22 or h < 7) else 0)

# 3. Sequence Features 재계산 (중간에 끼어든 Lab 이벤트 포함하여 순서 다시 매김)
df_final['prefix_len'] = df_final.groupby('hadm_id').cumcount() + 1

# 4. Count & State Features 재계산
# (단순 ffill로는 중간에 끼어든 Lab 이벤트의 count가 갱신 안 될 수 있으므로 재계산 추천)
df_final['is_ecg'] = (df_final['event_name'] == 'ECG_TAKEN').astype(int)
df_final['cum_ecg_cnt'] = df_final.groupby('hadm_id')['is_ecg'].cumsum()

df_final['is_trop'] = (df_final['event_name'] == 'TROP_TAKEN').astype(int)
df_final['cum_trop_cnt'] = df_final.groupby('hadm_id')['is_trop'].cumsum()

# Flag 재계산
mask_stemi = (df_final['first_stemi_ecg_time'].notna()) & (df_final['current_time'] >= df_final['first_stemi_ecg_time'])
df_final['stemi_flag'] = mask_stemi.astype(int)

mask_trop_pos = (df_final['first_troponin_positive_charttime'].notna()) & (df_final['current_time'] >= df_final['first_troponin_positive_charttime'])
df_final['trop_pos_flag'] = mask_trop_pos.astype(int)

# Pathway Stage & ID 매핑 재계산
stage_map = {
    'ED_ARRIVAL': 0, 'ECG_TAKEN': 1, 'TROP_TAKEN': 1,
    'STEMI_FLAG': 2, 'ANTI_TAKEN': 3,
    'PCI_START': 4, 'PCI_END': 5, 'DISCHARGE': 5
}
df_final['pathway_stage'] = df_final['event_name'].map(stage_map).fillna(0)

event_id_map = {name: i for i, name in enumerate(stage_map.keys())}
df_final['current_event_id'] = df_final['event_name'].map(event_id_map)

# PCI Status 재계산 (forward fill 방식)
df_final['pci_status'] = 0
df_final.loc[df_final['event_name'] == 'PCI_START', 'pci_status'] = 1
df_final.loc[df_final['event_name'] == 'PCI_END', 'pci_status'] = 2
# [수정 전 코드 - 에러 발생]
# df_final['pci_status'] = df_final.groupby('hadm_id')['pci_status'].replace(0, method='ffill').fillna(0)

# [수정 후 코드]
# 1. 0을 NaN으로 변경 (ffill은 NaN을 채우는 함수이므로)
df_final['pci_status'] = df_final['pci_status'].replace(0, np.nan)

# 2. 그룹별 Forward Fill (이전 상태를 현재로 가져오기)
# 예: 1(Start) -> NaN -> NaN -> 2(End)  ==>  1 -> 1 -> 1 -> 2
df_final['pci_status'] = df_final.groupby('hadm_id')['pci_status'].ffill()

# 3. 여전히 NaN인 값(PCI 시작 전이라 앞의 값이 없는 경우)을 0으로 원복
df_final['pci_status'] = df_final['pci_status'].fillna(0).astype(int)
# 주의: PCI_END 이후에는 2가 유지되어야 하고, START 이전엔 0이어야 함.
# 위 단순 ffill은 START(1) -> END(2) 전이에는 문제없으나,
# 엄밀하게는 "현재 시점이 pci_endtime 이후인가?" 로직이 더 정확할 수 있음.
# 일단 위 로직으로도 대부분 커버 가능.

print("동적 피처 재계산(Repair) 완료.")

동적 피처 재계산(Repair) 완료.


In [ ]:
# =============================================================================
# [Step 4.5] 병합 후 피처 재계산 (Repair Missing Values)
# =============================================================================
# 설명: Lab 데이터(df_trop)가 병합되면서 비어있는 ed_intime과 동적 피처들을 다시 채웁니다.

# 1. ed_intime 및 주요 시간 정보 채우기
# hadm_id별로 ed_intime이 같은 환자라면 동일하므로 forward fill/backward fill로 채웁니다.
time_fill_cols = ['ed_intime', 'first_stemi_ecg_time', 'first_troponin_positive_charttime']
for col in time_fill_cols:
    if col in df_final.columns:
        df_final[col] = df_final.groupby('hadm_id')[col].transform('first') # 그룹 내 첫 번째 값으로 채움

# 2. Time Features 재계산 (ed_intime이 채워졌으므로 가능)
df_final['current_time'] = df_final['charttime']
df_final['time_since_ed'] = (df_final['current_time'] - df_final['ed_intime']).dt.total_seconds() / 60

# prev_event_time & time_since_last 재계산 (새로 들어온 Lab 이벤트 순서 반영)
df_final = df_final.sort_values(by=['subject_id', 'hadm_id', 'charttime']) # 시간순 정렬 필수
df_final['prev_event_time'] = df_final.groupby('hadm_id')['charttime'].shift(1)
df_final['time_since_last'] = (df_final['current_time'] - df_final['prev_event_time']).dt.total_seconds() / 60
df_final['time_since_last'] = df_final['time_since_last'].fillna(0)

# is_night 재계산
df_final['hour'] = df_final['current_time'].dt.hour
df_final['is_night'] = df_final['hour'].apply(lambda h: 1 if (h >= 22 or h < 7) else 0)

# 3. Sequence Features 재계산 (중간에 끼어든 Lab 이벤트 포함하여 순서 다시 매김)
df_final['prefix_len'] = df_final.groupby('hadm_id').cumcount() + 1

# 4. Count & State Features 재계산
# (단순 ffill로는 중간에 끼어든 Lab 이벤트의 count가 갱신 안 될 수 있으므로 재계산 추천)
df_final['is_ecg'] = (df_final['event_name'] == 'ECG_TAKEN').astype(int)
df_final['cum_ecg_cnt'] = df_final.groupby('hadm_id')['is_ecg'].cumsum()

df_final['is_trop'] = (df_final['event_name'] == 'TROP_TAKEN').astype(int)
df_final['cum_trop_cnt'] = df_final.groupby('hadm_id')['is_trop'].cumsum()

# Flag 재계산
mask_stemi = (df_final['first_stemi_ecg_time'].notna()) & (df_final['current_time'] >= df_final['first_stemi_ecg_time'])
df_final['stemi_flag'] = mask_stemi.astype(int)

mask_trop_pos = (df_final['first_troponin_positive_charttime'].notna()) & (df_final['current_time'] >= df_final['first_troponin_positive_charttime'])
df_final['trop_pos_flag'] = mask_trop_pos.astype(int)

# Pathway Stage & ID 매핑 재계산
stage_map = {
    'ED_ARRIVAL': 0, 'ECG_TAKEN': 1, 'TROP_TAKEN': 1,
    'STEMI_FLAG': 2, 'ANTI_TAKEN': 3,
    'PCI_START': 4, 'PCI_END': 5, 'DISCHARGE': 5
}
df_final['pathway_stage'] = df_final['event_name'].map(stage_map).fillna(0)

event_id_map = {name: i for i, name in enumerate(stage_map.keys())}
df_final['current_event_id'] = df_final['event_name'].map(event_id_map)

import numpy as np # np.nan 사용을 위해 필요

# PCI Status 재계산 (수정됨)
df_final['pci_status'] = 0
df_final.loc[df_final['event_name'] == 'PCI_START', 'pci_status'] = 1
df_final.loc[df_final['event_name'] == 'PCI_END', 'pci_status'] = 2

# [핵심 수정] 0을 NaN으로 변경 -> 그룹별 ffill -> 다시 0으로 채우기
# 1. 0을 NaN으로 변경 (ffill은 NaN 값을 앞의 값으로 채우는 함수이므로)
df_final['pci_status'] = df_final['pci_status'].replace(0, np.nan)

# 2. 그룹별 Forward Fill 적용
# (이전 시점의 상태 1(진행중) 또는 2(종료)를 현재 시점(0이었던 곳)으로 전파)
df_final['pci_status'] = df_final.groupby('hadm_id')['pci_status'].ffill()

# 3. 여전히 NaN인 값(PCI 시작 전이라 앞의 값이 없는 경우)을 0으로 원복하고 정수형 변환
df_final['pci_status'] = df_final['pci_status'].fillna(0).astype(int)

# ... (이후 print 문 등)

print("동적 피처 재계산(Repair) 완료.")

동적 피처 재계산(Repair) 완료.


In [ ]:
import numpy as np
import pandas as pd

# -----------------------------------------------------------
# [설정] EOS(End of Sequence) 토큰 정의
# -----------------------------------------------------------
# df_final['current_event_id']가 이미 존재한다고 가정합니다.
EOS_TOKEN = int(df_final['current_event_id'].max()) + 1

# -----------------------------------------------------------
# 1. Delay Features (Raw Values)
# -----------------------------------------------------------
# 1-1. 각 환자의 '첫 이벤트 발생 시각' 테이블 만들기
first_times = df_final.groupby(['hadm_id', 'event_name'])['charttime'].min().unstack()

# 타겟 이벤트 정의
target_events = {
    'ECG_TAKEN': 'delay_ecg',
    'TROP_TAKEN': 'delay_trop',
    'ANTI_TAKEN': 'delay_med' # 실제 이벤트명: ANTI_PLT_ADMIN 등 확인 필요
}

# 병합을 위해 ed_intime 확보
if 'ed_intime' not in df_final.columns:
    df_final = df_final.merge(df_cohort[['hadm_id', 'ed_intime']], on='hadm_id', how='left')

# first_times 정보를 df_final에 병합
temp_delay = df_final[['hadm_id', 'current_time', 'ed_intime']].merge(
    first_times.rename(columns=lambda x: f"first_{x}_time"),
    on='hadm_id',
    how='left'
)

for evt_name, col_name in target_events.items():
    first_col = f"first_{evt_name}_time"

    if first_col in temp_delay.columns:
        # 계산된 지연 시간 (분 단위 Raw Value)
        fixed_delay = (temp_delay[first_col] - temp_delay['ed_intime']).dt.total_seconds() / 60

        # 조건: 이벤트 기록이 없거나(NaT), 현재 시각이 첫 이벤트보다 빠르면 -> -1
        # 그 외 -> fixed_delay 그대로 사용 (로그 변환 X)
        df_final[col_name] = np.where(
            (temp_delay[first_col].isna()) | (temp_delay['current_time'] < temp_delay[first_col]),
            -1,
            fixed_delay
        )
    else:
        df_final[col_name] = -1

# -----------------------------------------------------------
# 2. Cyclic Time Encoding (Feature Generation)
# -----------------------------------------------------------
# 이건 스케일링이라기보다 '새로운 정보 생성'이므로 유지하는 것을 추천합니다.
# (모델이 시간을 이해하는 데 필수적임)
df_final['hour_float'] = df_final['current_time'].dt.hour + df_final['current_time'].dt.minute / 60.0
df_final['sin_time'] = np.sin(2 * np.pi * df_final['hour_float'] / 24.0)
df_final['cos_time'] = np.cos(2 * np.pi * df_final['hour_float'] / 24.0)

# -----------------------------------------------------------
# 3. Target Generation (Shift -1)
# -----------------------------------------------------------
# [Target 1] Next Event ID (다음 행의 ID, 없으면 EOS)
df_final['target_next_evt'] = df_final.groupby('hadm_id')['current_event_id'].shift(-1).fillna(EOS_TOKEN).astype(int)

# [Target 2] Time to Next Event (분 단위 Raw Value)
next_time = df_final.groupby('hadm_id')['current_time'].shift(-1)
df_final['target_time_to_next'] = (next_time - df_final['current_time']).dt.total_seconds() / 60
df_final['target_time_to_next'] = df_final['target_time_to_next'].fillna(0)
# (로그 변환 제거됨)

# [Target 3] Remain LOS (시간 단위 Raw Value)
if 'dischtime' not in df_final.columns:
     df_final = df_final.merge(df_cohort[['hadm_id', 'dischtime']], on='hadm_id', how='left')

df_final['target_remain_los'] = (df_final['dischtime'] - df_final['current_time']).dt.total_seconds() / 3600
df_final['target_remain_los'] = df_final['target_remain_los'].fillna(0)
df_final.loc[df_final['target_remain_los'] < 0, 'target_remain_los'] = 0

# [Target 4] Mortality (Binary)
if 'death_flag' not in df_final.columns:
    df_final = df_final.merge(df_cohort[['hadm_id', 'death_flag']], on='hadm_id', how='left')
df_final['target_mortality'] = df_final['death_flag'].fillna(0).astype(int)

# -----------------------------------------------------------
# 4. Final Master Table Construction (수정됨)
# -----------------------------------------------------------
final_columns = [
    # Key (3개)
    'subject_id', 'hadm_id', 'prefix_len',

    # Static Features (6개) - ★여기가 빠졌었습니다!
    'age', 'gender', 'race', 'arrival_transport_code',
    'cci_score', 'hfrs_score',

    # Dynamic - Sequence (1개)
    'current_event_id',

    # Dynamic - Time (Raw) (4개)
    'time_since_ed', 'sin_time', 'cos_time', 'is_night',

    # Dynamic - Delay (Raw) (3개)
    'delay_ecg', 'delay_trop', 'delay_med',

    # Dynamic - State (4개)
    'stemi_flag', 'trop_pos_flag', 'pci_status', 'pathway_stage',

    # Dynamic - Lab (Raw) (5개)
    'last_trop', 'run_max_trop', 'trop_trend',
    'cum_ecg_cnt', 'cum_trop_cnt',

    # Targets (4개)
    'target_next_evt', 'target_time_to_next', 'target_remain_los', 'target_mortality'
]

# (중요) 정적 컬럼들이 df_final에 있는지 확인하고, 없으면 코호트에서 merge해야 합니다.
static_cols_to_merge = ['age', 'gender', 'race', 'cci_score', 'hfrs_score', 'arrival_transport_code']
missing_static = [c for c in static_cols_to_merge if c not in df_final.columns]

if missing_static:
    print(f"Merge missing static cols: {missing_static}")
    df_final = df_final.merge(df_cohort[['hadm_id'] + missing_static], on='hadm_id', how='left')

# 최종 선택
existing_cols = [c for c in final_columns if c in df_final.columns]
df_master = df_final[existing_cols].copy()

print(f"최종 컬럼 개수: {len(df_master.columns)}")
print(df_master.columns.tolist())
df_master

최종 컬럼 개수: 30
['subject_id', 'hadm_id', 'prefix_len', 'age', 'gender', 'race', 'arrival_transport_code', 'cci_score', 'hfrs_score', 'current_event_id', 'time_since_ed', 'sin_time', 'cos_time', 'is_night', 'delay_ecg', 'delay_trop', 'delay_med', 'stemi_flag', 'trop_pos_flag', 'pci_status', 'pathway_stage', 'last_trop', 'run_max_trop', 'trop_trend', 'cum_ecg_cnt', 'cum_trop_cnt', 'target_next_evt', 'target_time_to_next', 'target_remain_los', 'target_mortality']


,subject_id,hadm_id,prefix_len,age,gender,race,arrival_transport_code,cci_score,hfrs_score,current_event_id,...,pathway_stage,last_trop,run_max_trop,trop_trend,cum_ecg_cnt,cum_trop_cnt,target_next_evt,target_time_to_next,target_remain_los,target_mortality
0,10000764,27897940,1,86,M,WHITE,0,0,0.0,0,...,0,0.00,0.00,0.00,0,0,1,9.0,116.983333,0
1,10000764,27897940,2,86,M,WHITE,0,0,0.0,1,...,1,0.00,0.00,0.00,1,0,3,0.0,116.833333,0
2,10000764,27897940,3,86,M,WHITE,0,0,0.0,3,...,2,0.00,0.00,0.00,1,0,2,725.0,116.833333,0
3,10000764,27897940,4,86,M,WHITE,0,0,0.0,2,...,1,0.04,0.04,0.04,1,1,4,735.0,104.750000,0
4,10000764,27897940,5,86,M,WHITE,0,0,0.0,4,...,3,0.04,0.04,0.00,1,1,2,603.0,92.500000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14307,19996783,21880161,2,89,M,ASIAN - CHINESE,1,1,15.1,0,...,0,0.00,0.00,0.00,1,0,2,1014.0,253.116667,1
14308,19996783,21880161,3,89,M,ASIAN - CHINESE,1,1,15.1,2,...,1,1.37,1.37,1.37,1,1,4,64.0,236.216667,1
14309,19996783,21880161,4,89,M,ASIAN - CHINESE,1,1,15.1,4,...,3,1.37,1.37,0.00,1,1,2,296.0,235.150000,1
14310,19996783,21880161,5,89,M,ASIAN - CHINESE,1,1,15.1,2,...,1,1.92,1.92,0.55,1,2,7,13813.0,230.216667,1


# [4] 결측치 파악

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1. 결측치 개수 및 비율 확인 함수
def check_missing_values(df):
    missing = df.isnull().sum()
    missing = missing[missing > 0] # 결측치 있는 것만 추출
    if missing.empty:
        print("🎉 결측치가 없습니다! 완벽합니다.")
        return None

    missing_df = pd.DataFrame({
        'Missing Count': missing,
        'Percentage': (missing / len(df)) * 100
    })
    missing_df = missing_df.sort_values(by='Percentage', ascending=False)

    print("=== [EDA] 컬럼별 결측치 현황 ===")
    print(missing_df)

    return missing_df

# 실행
missing_report = check_missing_values(df_master)

🎉 결측치가 없습니다! 완벽합니다.


In [ ]:
import os

# 1. 저장할 파일명 및 경로 설정
file_name = 'cohort_ver121_features.csv'
save_path = os.path.join(base_path, file_name)

# 2. CSV로 저장
# index=False: 불필요한 행 번호 저장 안 함
df_master.to_csv(save_path, index=False)

print(f"파일 저장 완료!")
print(f"저장 경로: {save_path}")
print(f"데이터 크기: {df_master.shape}")

파일 저장 완료!
저장 경로: /content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/cohort_ver121_features.csv
데이터 크기: (14312, 30)


In [36]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler

# 1. 컬럼 정의 (PDF 인덱스 1~35 반영)
# PDF상 1-32는 피처/현재상태, 33-35는 예측 타겟입니다.
columns_map = {
    # --- Key & Demographics (1-5) ---
    1: 'subject_id',
    2: 'hadm_id',
    3: 'age',
    4: 'gender',          # 0:F, 1:M, -1:Unknown
    5: 'race',            # Label Encoding

    # --- Hospital/ED Info (6-8) ---
    6: 'arrival_transport', # 0:Unknown, 1:Walk-in, 2:Ambulance, 3:Helicopter
    7: 'cci_score',
    8: 'hfrs_score',

    # --- Time Delays (Defaults: -1) (9-12) ---
    9: 'door_to_ecg',
    10: 'door_to_trop',
    11: 'door_to_anti',
    12: 'door_to_pci',      # PDF상 'door to poi' (오타 수정 반영)

    # --- Temporal & Count Features (13-19) ---
    13: 'prefix_len',       # Sequence Length
    14: 'time_since_ed',
    15: 'time_since_last',
    16: 'is_night',         # 0 or 1
    17: 'cum_ecg_cnt',
    18: 'cum_stemi_cnt',
    19: 'cum_trop_cnt',

    # --- Flags & Values (20-25) ---
    20: 'stemi_flag',
    21: 'trop_pos_flag',
    22: 'last_trop',        # Standard Scaling 필요
    23: 'run_max_trop',     # Standard Scaling 필요
    24: 'trop_trend',
    25: 'pci_status',       # PDF상 'poi status' (0:수행전, 1:진행, 2:종료)

    # --- Context & State (26-31) ---
    26: 'pathway_stage',
    27: 'time_since_start_min',
    28: 'current_event_id', # Integer Mapping
    29: 'prefix_events_str',
    30: 'current_heart_rate',
    31: 'current_mean_bp',

    # --- Targets (32-35) ---
    32: 'target_mortality', # Binary (0/1)
    33: 'target_next_evt',  # Next Event ID (EOS 포함)
    34: 'target_time_to_next',
    35: 'target_remain_los'
}

def preprocess_cohort_data(df):
    """
    PDF 명세에 따른 데이터 전처리 함수
    """
    df_processed = df.copy()

    # ---------------------------------------------------------
    # 1. Event ID 매핑 및 EOS 토큰 추가 (요청사항 1번)
    # ---------------------------------------------------------
    # 실제 데이터에 있는 이벤트 목록 추출
    unique_events = df_processed['event_name'].unique().tolist() if 'event_name' in df_processed.columns else ['event_A', 'event_B'] # 예시

    # EOS 토큰 추가
    if 'EOS' not in unique_events:
        unique_events.append('EOS')

    # 매핑 딕셔너리 생성
    event_to_idx = {evt: idx for idx, evt in enumerate(unique_events)}
    print(f"✅ Event Vocabulary Size (with EOS): {len(event_to_idx)}")
    print(f"✅ EOS Token ID: {event_to_idx['EOS']}")

    # current_event_id 매핑 (예시)
    if 'event_name' in df_processed.columns:
        df_processed['current_event_id'] = df_processed['event_name'].map(event_to_idx)

    # ---------------------------------------------------------
    # 2. 결측치 및 Default 값 처리 (PDF 명세 기반)
    # ---------------------------------------------------------
    # Delay 관련 (-1로 채움)
    delay_cols = ['door_to_ecg', 'door_to_trop', 'door_to_anti', 'door_to_pci']
    for col in delay_cols:
        if col in df_processed.columns:
            df_processed[col] = df_processed[col].fillna(-1)

    # Count/Time 관련 (0으로 채움)
    zero_fill_cols = ['cum_ecg_cnt', 'cum_stemi_cnt', 'cum_trop_cnt',
                      'time_since_ed', 'time_since_last', 'trop_trend']
    for col in zero_fill_cols:
        if col in df_processed.columns:
            df_processed[col] = df_processed[col].fillna(0)

    # ---------------------------------------------------------
    # 3. 인코딩 및 스케일링
    # ---------------------------------------------------------
    # Gender (0:F, 1:M) - 데이터 상황에 맞춰 매핑 필요
    if 'gender' in df_processed.columns:
        # 예: 'F'->0, 'M'->1, Others->-1
        df_processed['gender'] = df_processed['gender'].map({'F': 0, 'M': 1}).fillna(-1)

    # Arrival Transport (Label Encoding: 0~3)
    # PDF: 0(unknown)/1(walkin)/2(ambulance)/3(helicopter)
    if 'arrival_transport' in df_processed.columns:
        transport_map = {'unknown': 0, 'walkin': 1, 'ambulance': 2, 'helicopter': 3}
        # 텍스트로 되어있다면 매핑, 이미 숫자라면 유지
        if df_processed['arrival_transport'].dtype == 'O':
             df_processed['arrival_transport'] = df_processed['arrival_transport'].map(transport_map).fillna(0)

    # Standard Scaling (last_trop, run_max_trop)
    scaler = StandardScaler()
    scale_cols = ['last_trop', 'run_max_trop']
    for col in scale_cols:
        if col in df_processed.columns:
            # 결측치는 평균(0) 혹은 특정 값으로 채운 후 스케일링 권장 (여기선 0 처리 후 진행 예시)
            df_processed[col] = df_processed[col].fillna(0)
            df_processed[col] = scaler.fit_transform(df_processed[[col]])

    # ---------------------------------------------------------
    # 4. Target 생성 로직 예시 (EOS 처리)
    # ---------------------------------------------------------
    # 시퀀스 데이터라면 groupby 후 shift(-1) 등을 사용해 다음 이벤트를 가져옵니다.
    # 여기서는 예시 로직입니다.
    # df_processed['target_next_evt'] = df_processed.groupby('hadm_id')['current_event_id'].shift(-1)
    # df_processed['target_next_evt'] = df_processed['target_next_evt'].fillna(event_to_idx['EOS']) # 마지막은 EOS

    return df_processed, event_to_idx

# --- 실행 예시 (Dummy Data) ---
data = {
    'hadm_id': [101, 101, 102],
    'event_name': ['ED Triage', 'ECG', 'ED Triage'],
    'gender': ['M', 'M', 'F'],
    'arrival_transport': ['ambulance', 'ambulance', 'walkin'],
    'last_trop': [0.01, 0.05, 0.02],
    'run_max_trop': [0.01, 0.05, 0.02],
    'door_to_pci': [None, None, 15.0] # None은 -1이 되어야 함
}
df = pd.DataFrame(data)

# 전처리 실행
processed_df, event_vocab = preprocess_cohort_data(df)

# 결과 확인
print("\n[전처리 후 데이터 샘플]")
print(processed_df[['hadm_id', 'current_event_id', 'door_to_pci', 'arrival_transport']].head())
print("\n[스케일링 확인]")
print(processed_df[['last_trop']])

✅ Event Vocabulary Size (with EOS): 3
✅ EOS Token ID: 2

[전처리 후 데이터 샘플]
   hadm_id  current_event_id  door_to_pci  arrival_transport
0      101                 0         -1.0                  2
1      101                 1         -1.0                  2
2      102                 0         15.0                  1

[스케일링 확인]
   last_trop
0  -0.980581
1   1.372813
2  -0.392232


In [47]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

def get_cohort_schema():
    """
    PDF 문서의 1~35번 인덱스에 해당하는 컬럼 정의를 반환합니다.
    """
    return [
        # --- Key & Demographics (1-5) ---
        'subject_id', 'hadm_id', 'age', 'gender', 'race',
        # --- Hospital/ED Info (6-8) ---
        'arrival_transport', 'cci_score', 'hfrs_score',
        # --- Time Delays (Defaults: -1) (9-12) ---
        'door_to_ecg', 'door_to_trop', 'door_to_anti', 'door_to_pci',
        # --- Temporal & Count Features (13-19) ---
        'prefix_len', 'time_since_ed', 'time_since_last', 'is_night',
        'cum_ecg_cnt', 'cum_stemi_cnt', 'cum_trop_cnt',
        # --- Flags & Values (20-25) ---
        'stemi_flag', 'trop_pos_flag', 'last_trop', 'run_max_trop', 'trop_trend', 'pci_status',
        # --- Context & State (26-31) ---
        'pathway_stage', 'time_since_start_min', 'current_event_id', 'prefix_events_str',
        'current_heart_rate', 'current_mean_bp',
        # --- Targets (32-35) ---
        'target_mortality', 'target_next_evt', 'target_time_to_next', 'target_remain_los'
    ]

def preprocess_cohort_data(df):
    """
    수정된 요청사항(Troponin -1 처리, EOS 14, Padding 0)을 반영한 전처리 함수
    """
    df_processed = df.copy()

    # =========================================================
    # 상수 정의 (Constants)
    # =========================================================
    EOS_TOKEN_ID = 14  # End of Sequence
    PAD_TOKEN_ID = 0   # Padding (참고용, 실제 패딩은 배치는 단계에서 수행)

    # ---------------------------------------------------------
    # 1. 결측치(NaN) 및 Default 값 처리
    # ---------------------------------------------------------
    # Delay 관련: 미수행 시 -1
    delay_cols = ['door_to_ecg', 'door_to_trop', 'door_to_anti', 'door_to_pci']
    for col in delay_cols:
        if col in df_processed.columns:
            df_processed[col] = df_processed[col].fillna(-1)

    # Count/Time 관련: 없음 시 0
    zero_fill_cols = [
        'cum_ecg_cnt', 'cum_stemi_cnt', 'cum_trop_cnt',
        'time_since_ed', 'time_since_last', 'trop_trend',
        'current_heart_rate', 'current_mean_bp', 'time_since_start_min'
    ]
    for col in zero_fill_cols:
        if col in df_processed.columns:
            df_processed[col] = df_processed[col].fillna(0)

    # ---------------------------------------------------------
    # 2. 범주형 데이터 인코딩
    # ---------------------------------------------------------
    # Gender (예: M->1, F->0)
    if 'gender' in df_processed.columns and df_processed['gender'].dtype == 'O':
        df_processed['gender'] = df_processed['gender'].map({'M': 1, 'F': 0}).fillna(-1)

    # Arrival Transport
    if 'arrival_transport' in df_processed.columns and df_processed['arrival_transport'].dtype == 'O':
        transport_map = {'unknown': 0, 'walkin': 1, 'ambulance': 2, 'helicopter': 3}
        df_processed['arrival_transport'] = df_processed['arrival_transport'].map(transport_map).fillna(0)

    # ---------------------------------------------------------
    # 3. 스케일링 (Standard Scaling) - 수정됨
    # ---------------------------------------------------------
    # Troponin 검사를 안 했으면 -1, 했으면(0 포함) Standard Scaling
    scaler = StandardScaler()
    scale_cols = ['last_trop', 'run_max_trop']

    for col in scale_cols:
        if col in df_processed.columns:
            # (1) 유효한 값(NaN이 아닌 값) 마스킹
            valid_mask = df_processed[col].notna()

            # (2) 유효한 데이터가 있다면 스케일링 수행
            if valid_mask.any():
                # 2차원 배열 형태로 변환하여 fit_transform
                valid_data = df_processed.loc[valid_mask, [col]]
                df_processed.loc[valid_mask, col] = scaler.fit_transform(valid_data)

            # (3) 나머지(NaN, 검사 안 함)는 -1로 채움
            df_processed[col] = df_processed[col].fillna(-1)

    # ---------------------------------------------------------
    # 4. Target 생성: Next Event 및 EOS 처리 - 수정됨
    # ---------------------------------------------------------
    # 다음 이벤트 ID 생성 (그룹별 Shift)
    if 'current_event_id' in df_processed.columns:
        df_processed['target_next_evt'] = df_processed.groupby('hadm_id')['current_event_id'].shift(-1)

        # 마지막 이벤트(Shift 결과가 NaN)인 경우 EOS_TOKEN_ID(14) 할당
        df_processed['target_next_evt'] = df_processed['target_next_evt'].fillna(EOS_TOKEN_ID).astype(int)

    # 다음 이벤트까지 시간
    if 'time_since_start_min' in df_processed.columns:
        next_time = df_processed.groupby('hadm_id')['time_since_start_min'].shift(-1)
        df_processed['target_time_to_next'] = next_time - df_processed['time_since_start_min']
        df_processed['target_time_to_next'] = df_processed['target_time_to_next'].fillna(0)

    # ---------------------------------------------------------
    # 5. 컬럼 순서 정렬 및 마무리
    # ---------------------------------------------------------
    schema_cols = get_cohort_schema()
    for col in schema_cols:
        if col not in df_processed.columns:
            df_processed[col] = 0 # 없는 컬럼 0으로 초기화

    return df_processed[schema_cols]

# ==========================================
# 실행 예시 (테스트)
# ==========================================
raw_data = {
    'subject_id': [1001, 1001, 1001, 1002],
    'hadm_id': [2001, 2001, 2001, 2002],
    'current_event_id': [10, 11, 12, 10],   # 예시 이벤트 ID
    'last_trop': [0.05, None, 0.0, None],   # 0.05(값 있음), NaN(안함), 0.0(값 있음), NaN
    'run_max_trop': [0.05, None, 0.05, None],
    'time_since_start_min': [0, 10, 20, 0]
}

df_raw = pd.DataFrame(raw_data)
feature_matrix = preprocess_cohort_data(df_raw)

print("=== 수정된 로직 확인 ===")
print("1. Troponin 스케일링 확인 (-1은 미수행, 나머지는 스케일링):")
print(feature_matrix[['last_trop']].T)
print("\n2. EOS 토큰(14) 확인 (target_next_evt):")
feature_matrix

=== 수정된 로직 확인 ===
1. Troponin 스케일링 확인 (-1은 미수행, 나머지는 스케일링):
             0    1    2    3
last_trop  1.0 -1.0 -1.0 -1.0

2. EOS 토큰(14) 확인 (target_next_evt):


,subject_id,hadm_id,age,gender,race,arrival_transport,cci_score,hfrs_score,door_to_ecg,door_to_trop,...,pathway_stage,time_since_start_min,current_event_id,prefix_events_str,current_heart_rate,current_mean_bp,target_mortality,target_next_evt,target_time_to_next,target_remain_los
0,1001,2001,0,0,0,0,0,0,0,0,...,0,0,10,0,0,0,0,11,10.0,0
1,1001,2001,0,0,0,0,0,0,0,0,...,0,10,11,0,0,0,0,12,10.0,0
2,1001,2001,0,0,0,0,0,0,0,0,...,0,20,12,0,0,0,0,14,0.0,0
3,1002,2002,0,0,0,0,0,0,0,0,...,0,0,10,0,0,0,0,14,0.0,0


In [49]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# 1. 스키마 정의 함수
def get_cohort_schema():
    return [
        # --- Key & Demographics ---
        'subject_id', 'hadm_id', 'age', 'gender', 'race',
        # --- Hospital/ED Info ---
        'arrival_transport', 'cci_score', 'hfrs_score',
        # --- Time Delays (Default: -1) ---
        'door_to_ecg', 'door_to_trop', 'door_to_anti', 'door_to_pci',
        # --- Temporal & Count Features ---
        'prefix_len', 'time_since_ed', 'time_since_last', 'is_night',
        'cum_ecg_cnt', 'cum_stemi_cnt', 'cum_trop_cnt',
        # --- Flags & Values ---
        'stemi_flag', 'trop_pos_flag', 'last_trop', 'run_max_trop', 'trop_trend', 'pci_status',
        # --- Context & State ---
        'pathway_stage', 'time_since_start_min', 'current_event_id', 'prefix_events_str',
        'current_heart_rate', 'current_mean_bp',
        # --- Targets ---
        'target_mortality', 'target_next_evt', 'target_time_to_next', 'target_remain_los'
    ]

# 2. 전처리 핵심 로직
def preprocess_cohort_data(df):
    # 원본 데이터 보존을 위해 복사
    df_processed = df.copy()

    # --- 상수 정의 ---
    EOS_TOKEN_ID = 14  # End of Sequence 토큰

    # ---------------------------------------------------------
    # A. 결측치(NaN) 및 Default 값 처리
    # ---------------------------------------------------------
    # Delay 관련 (-1로 채움)
    delay_cols = ['door_to_ecg', 'door_to_trop', 'door_to_anti', 'door_to_pci']
    for col in delay_cols:
        if col in df_processed.columns:
            df_processed[col] = df_processed[col].fillna(-1)

    # Count/Time 관련 (0으로 채움)
    zero_fill_cols = [
        'cum_ecg_cnt', 'cum_stemi_cnt', 'cum_trop_cnt',
        'time_since_ed', 'time_since_last', 'trop_trend',
        'current_heart_rate', 'current_mean_bp', 'time_since_start_min'
    ]
    for col in zero_fill_cols:
        if col in df_processed.columns:
            df_processed[col] = df_processed[col].fillna(0)

    # ---------------------------------------------------------
    # B. 범주형 데이터 인코딩
    # ---------------------------------------------------------
    # Gender (M->1, F->0)
    if 'gender' in df_processed.columns and df_processed['gender'].dtype == 'O':
        df_processed['gender'] = df_processed['gender'].map({'M': 1, 'F': 0}).fillna(-1)

    # Arrival Transport
    if 'arrival_transport' in df_processed.columns and df_processed['arrival_transport'].dtype == 'O':
        transport_map = {'unknown': 0, 'walkin': 1, 'ambulance': 2, 'helicopter': 3}
        df_processed['arrival_transport'] = df_processed['arrival_transport'].map(transport_map).fillna(0)

    # ---------------------------------------------------------
    # C. 스케일링 (Standard Scaling + NaN Handling)
    # ---------------------------------------------------------
    scaler = StandardScaler()
    scale_cols = ['last_trop', 'run_max_trop']

    for col in scale_cols:
        if col in df_processed.columns:
            # 유효한 값(NaN 아님)만 필터링
            valid_mask = df_processed[col].notna()

            # 유효한 값이 존재할 때만 스케일링 수행
            if valid_mask.any():
                valid_data = df_processed.loc[valid_mask, [col]]
                df_processed.loc[valid_mask, col] = scaler.fit_transform(valid_data)

            # 나머지(NaN)는 -1로 채움 (검사 미수행)
            df_processed[col] = df_processed[col].fillna(-1)

    # ---------------------------------------------------------
    # D. Target 생성 (EOS 및 Next Event)
    # ---------------------------------------------------------
    # 1. Next Event ID (마지막은 EOS=14)
    if 'current_event_id' in df_processed.columns:
        # 입원(hadm_id)별로 다음 이벤트 ID를 가져옴
        df_processed['target_next_evt'] = df_processed.groupby('hadm_id')['current_event_id'].shift(-1)
        # NaN(마지막 순서)을 EOS 토큰으로 채움
        df_processed['target_next_evt'] = df_processed['target_next_evt'].fillna(EOS_TOKEN_ID).astype(int)

    # 2. Time to Next Event
    if 'time_since_start_min' in df_processed.columns:
        next_time = df_processed.groupby('hadm_id')['time_since_start_min'].shift(-1)
        df_processed['target_time_to_next'] = next_time - df_processed['time_since_start_min']
        df_processed['target_time_to_next'] = df_processed['target_time_to_next'].fillna(0)

    # ---------------------------------------------------------
    # E. 컬럼 정리 및 반환
    # ---------------------------------------------------------
    schema_cols = get_cohort_schema()
    for col in schema_cols:
        if col not in df_processed.columns:
            df_processed[col] = 0 # 없는 컬럼 0으로 초기화

    return df_processed[schema_cols]

# =========================================================
# [실행] 런타임에 있는 df_master 변수 사용
# =========================================================
try:
    # 1. 전처리 함수 실행 (입력: df_master)
    feature_matrix = preprocess_cohort_data(df_master)

    # 2. 결과 확인
    print(f"✅ 피쳐 매트릭스 생성 완료!")
    print(f"   - 원본 데이터 크기: {df_master.shape}")
    print(f"   - 생성된 매트릭스 크기: {feature_matrix.shape}")

except NameError:
    print("❌ 오류: 'df_master' 변수를 찾을 수 없습니다. 데이터를 먼저 로드해주세요.")
except Exception as e:
    print(f"❌ 오류 발생: {e}")

feature_matrix

✅ 피쳐 매트릭스 생성 완료!
   - 원본 데이터 크기: (14312, 31)
   - 생성된 매트릭스 크기: (14312, 35)


,subject_id,hadm_id,age,gender,race,arrival_transport,cci_score,hfrs_score,door_to_ecg,door_to_trop,...,pathway_stage,time_since_start_min,current_event_id,prefix_events_str,current_heart_rate,current_mean_bp,target_mortality,target_next_evt,target_time_to_next,target_remain_los
0,10000764,27897940,86,1,WHITE,0,0,0.0,0,0,...,0,0,0,0,0,0,0,1,9.0,116.983333
1,10000764,27897940,86,1,WHITE,0,0,0.0,0,0,...,1,0,1,0,0,0,0,3,0.0,116.833333
2,10000764,27897940,86,1,WHITE,0,0,0.0,0,0,...,2,0,3,0,0,0,0,2,725.0,116.833333
3,10000764,27897940,86,1,WHITE,0,0,0.0,0,0,...,1,0,2,0,0,0,0,4,735.0,104.750000
4,10000764,27897940,86,1,WHITE,0,0,0.0,0,0,...,3,0,4,0,0,0,0,2,603.0,92.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14307,19996783,21880161,89,1,ASIAN - CHINESE,0,1,15.1,0,0,...,0,0,0,0,0,0,1,2,1014.0,253.116667
14308,19996783,21880161,89,1,ASIAN - CHINESE,0,1,15.1,0,0,...,1,0,2,0,0,0,1,4,64.0,236.216667
14309,19996783,21880161,89,1,ASIAN - CHINESE,0,1,15.1,0,0,...,3,0,4,0,0,0,1,2,296.0,235.150000
14310,19996783,21880161,89,1,ASIAN - CHINESE,0,1,15.1,0,0,...,1,0,2,0,0,0,1,7,13813.0,230.216667


In [51]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# 1. 스키마 정의 함수
def get_cohort_schema():
    return [
        'subject_id', 'hadm_id', 'age', 'gender', 'race',
        'arrival_transport', 'cci_score', 'hfrs_score',
        'door_to_ecg', 'door_to_trop', 'door_to_anti', 'door_to_pci',
        'prefix_len', 'time_since_ed', 'time_since_last', 'is_night',
        'cum_ecg_cnt', 'cum_stemi_cnt', 'cum_trop_cnt',
        'stemi_flag', 'trop_pos_flag', 'last_trop', 'run_max_trop', 'trop_trend', 'pci_status',
        'pathway_stage', 'time_since_start_min', 'current_event_id', 'prefix_events_str',
        'current_heart_rate', 'current_mean_bp',
        'target_mortality', 'target_next_evt', 'target_time_to_next', 'target_remain_los'
    ]

# 2. 전처리 핵심 로직
def preprocess_cohort_data(df):
    df_processed = df.copy()

    # --- 상수 정의 ---
    EOS_TOKEN_ID = 14

    # ---------------------------------------------------------
    # 0. 데이터 정렬 (Forward Fill을 위해 필수)
    # ---------------------------------------------------------
    # hadm_id와 시점(prefix_len 또는 time_since_start_min) 기준으로 정렬되어 있어야 합니다.
    # 만약 데이터가 이미 정렬되어 있다면 이 부분은 생략 가능합니다.
    if 'prefix_len' in df_processed.columns:
        df_processed = df_processed.sort_values(by=['hadm_id', 'prefix_len'])

    # ---------------------------------------------------------
    # A. Troponin Forward Fill (LOCF 적용) - [수정된 부분]
    # ---------------------------------------------------------
    # last_trop과 run_max_trop은 "현재 시점까지의 상태"이므로,
    # 현재 이벤트에서 측정이 없더라도 이전 값을 유지해야 함.
    ffill_cols = ['last_trop', 'run_max_trop']

    for col in ffill_cols:
        if col in df_processed.columns:
            # 그룹별로 이전 값을 현재 NaN에 채워넣음 (예: NaN, 9, NaN, 8 -> NaN, 9, 9, 8)
            df_processed[col] = df_processed.groupby('hadm_id')[col].ffill()

    # ---------------------------------------------------------
    # B. 결측치(NaN) 및 Default 값 처리
    # ---------------------------------------------------------
    # Delay 관련 (-1)
    delay_cols = ['door_to_ecg', 'door_to_trop', 'door_to_anti', 'door_to_pci']
    for col in delay_cols:
        if col in df_processed.columns:
            df_processed[col] = df_processed[col].fillna(-1)

    # Count/Time 관련 (0)
    zero_fill_cols = [
        'cum_ecg_cnt', 'cum_stemi_cnt', 'cum_trop_cnt',
        'time_since_ed', 'time_since_last', 'trop_trend',
        'current_heart_rate', 'current_mean_bp', 'time_since_start_min'
    ]
    for col in zero_fill_cols:
        if col in df_processed.columns:
            df_processed[col] = df_processed[col].fillna(0)

    # ---------------------------------------------------------
    # C. 범주형 데이터 인코딩
    # ---------------------------------------------------------
    if 'gender' in df_processed.columns and df_processed['gender'].dtype == 'O':
        df_processed['gender'] = df_processed['gender'].map({'M': 1, 'F': 0}).fillna(-1)

    if 'arrival_transport' in df_processed.columns and df_processed['arrival_transport'].dtype == 'O':
        transport_map = {'unknown': 0, 'walkin': 1, 'ambulance': 2, 'helicopter': 3}
        df_processed['arrival_transport'] = df_processed['arrival_transport'].map(transport_map).fillna(0)

    # ---------------------------------------------------------
    # D. 스케일링 (Standard Scaling + NaN Handling) - [수정된 부분]
    # ---------------------------------------------------------
    scaler = StandardScaler()
    scale_cols = ['last_trop', 'run_max_trop']

    for col in scale_cols:
        if col in df_processed.columns:
            # Forward Fill 후에도 NaN인 값은 "입원 후 아직 한 번도 검사 안 함"을 의미
            valid_mask = df_processed[col].notna()

            # 유효한 값(검사 이력 있음)만 스케일링
            if valid_mask.any():
                valid_data = df_processed.loc[valid_mask, [col]]
                df_processed.loc[valid_mask, col] = scaler.fit_transform(valid_data)

            # 나머지(아직 검사 안 함)는 -1로 채움
            df_processed[col] = df_processed[col].fillna(-1)

    # ---------------------------------------------------------
    # E. Target 생성 (EOS 및 Next Event)
    # ---------------------------------------------------------
    if 'current_event_id' in df_processed.columns:
        df_processed['target_next_evt'] = df_processed.groupby('hadm_id')['current_event_id'].shift(-1)
        df_processed['target_next_evt'] = df_processed['target_next_evt'].fillna(EOS_TOKEN_ID).astype(int)

    if 'time_since_start_min' in df_processed.columns:
        next_time = df_processed.groupby('hadm_id')['time_since_start_min'].shift(-1)
        df_processed['target_time_to_next'] = next_time - df_processed['time_since_start_min']
        df_processed['target_time_to_next'] = df_processed['target_time_to_next'].fillna(0)

    # ---------------------------------------------------------
    # F. 컬럼 정리 및 반환
    # ---------------------------------------------------------
    schema_cols = get_cohort_schema()
    for col in schema_cols:
        if col not in df_processed.columns:
            df_processed[col] = 0

    return df_processed[schema_cols]

# =========================================================
# [실행] 런타임에 있는 df_master 변수 사용
# =========================================================
try:
    feature_matrix = preprocess_cohort_data(df_master)
    print(f"✅ 피쳐 매트릭스 생성 완료! (Forward Fill 적용됨)")
    print(f"   - 데이터 크기: {feature_matrix.shape}")

    # Forward Fill 결과 확인을 위한 샘플 출력 (last_trop 변화 확인)
    print("\n[Forward Fill 검증: last_trop 변화 확인]")

except NameError:
    print("❌ 오류: 'df_master' 변수를 찾을 수 없습니다.")
except Exception as e:
    print(f"❌ 오류 발생: {e}")

feature_matrix

✅ 피쳐 매트릭스 생성 완료! (Forward Fill 적용됨)
   - 데이터 크기: (14312, 35)

[Forward Fill 검증: last_trop 변화 확인]


,subject_id,hadm_id,age,gender,race,arrival_transport,cci_score,hfrs_score,door_to_ecg,door_to_trop,...,pathway_stage,time_since_start_min,current_event_id,prefix_events_str,current_heart_rate,current_mean_bp,target_mortality,target_next_evt,target_time_to_next,target_remain_los
5056,13647833,20006266,40,1,ASIAN - SOUTH EAST ASIAN,0,0,0.0,0,0,...,1,0,1,0,0,0,0,4,27504819.0,458714.683333
5057,13647833,20006266,40,1,ASIAN - SOUTH EAST ASIAN,0,0,0.0,0,0,...,3,0,4,0,0,0,0,0,654.0,301.033333
5058,13647833,20006266,40,1,ASIAN - SOUTH EAST ASIAN,0,0,0.0,0,0,...,0,0,0,0,0,0,0,2,626.0,290.133333
5059,13647833,20006266,40,1,ASIAN - SOUTH EAST ASIAN,0,0,0.0,0,0,...,1,0,2,0,0,0,0,2,614.0,279.700000
5060,13647833,20006266,40,1,ASIAN - SOUTH EAST ASIAN,0,0,0.0,0,0,...,1,0,2,0,0,0,0,7,16168.0,269.466667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9727,16976054,29997425,70,1,WHITE,0,1,6.6,0,0,...,1,0,2,0,0,0,0,2,589.0,128.816667
9728,16976054,29997425,70,1,WHITE,0,1,6.6,0,0,...,1,0,2,0,0,0,0,2,120.0,119.000000
9729,16976054,29997425,70,1,WHITE,0,1,6.6,0,0,...,1,0,2,0,0,0,0,2,205.0,117.000000
9730,16976054,29997425,70,1,WHITE,0,1,6.6,0,0,...,1,0,2,0,0,0,0,7,6815.0,113.583333


In [53]:
import os

# 1. 저장할 파일명 및 경로 설정
file_name = 'cohort_ver121_features.csv'
save_path = os.path.join(base_path, file_name)

# 2. CSV로 저장
# index=False: 불필요한 행 번호 저장 안 함
feature_matrix.to_csv(save_path, index=False)

print(f"파일 저장 완료!")
print(f"저장 경로: {save_path}")
print(f"데이터 크기: {feature_matrix.shape}")

파일 저장 완료!
저장 경로: /content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/cohort_ver121_features.csv
데이터 크기: (14312, 35)


In [54]:
# 컬럼이 존재하는지, 존재한다면 어떤 값들이 들어있는지 확인
if 'arrival_transport' in df_master.columns:
    print("현재 값 종류:", df_master['arrival_transport'].unique())
else:
    print("컬럼이 존재하지 않습니다. 조인이 누락되었는지 확인해주세요.")

컬럼이 존재하지 않습니다. 조인이 누락되었는지 확인해주세요.


In [55]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# 1. 스키마 정의 함수
def get_cohort_schema():
    return [
        'subject_id', 'hadm_id', 'age', 'gender', 'race',
        'arrival_transport', 'cci_score', 'hfrs_score',
        'door_to_ecg', 'door_to_trop', 'door_to_anti', 'door_to_pci',
        'prefix_len', 'time_since_ed', 'time_since_last', 'is_night',
        'cum_ecg_cnt', 'cum_stemi_cnt', 'cum_trop_cnt',
        'stemi_flag', 'trop_pos_flag', 'last_trop', 'run_max_trop', 'trop_trend', 'pci_status',
        'pathway_stage', 'time_since_start_min', 'current_event_id', 'prefix_events_str',
        'current_heart_rate', 'current_mean_bp',
        'target_mortality', 'target_next_evt', 'target_time_to_next', 'target_remain_los'
    ]

# 2. 전처리 핵심 로직 (수정본)
def preprocess_cohort_data(df):
    df_processed = df.copy()
    EOS_TOKEN_ID = 14

    # ---------------------------------------------------------
    # 0. 데이터 정렬
    # ---------------------------------------------------------
    if 'prefix_len' in df_processed.columns:
        df_processed = df_processed.sort_values(by=['hadm_id', 'prefix_len'])

    # ---------------------------------------------------------
    # A. Forward Fill (LOCF)
    # ---------------------------------------------------------
    ffill_cols = ['last_trop', 'run_max_trop']
    for col in ffill_cols:
        if col in df_processed.columns:
            df_processed[col] = df_processed.groupby('hadm_id')[col].ffill()

    # ---------------------------------------------------------
    # B. 결측치(NaN) 처리
    # ---------------------------------------------------------
    delay_cols = ['door_to_ecg', 'door_to_trop', 'door_to_anti', 'door_to_pci']
    for col in delay_cols:
        if col in df_processed.columns:
            df_processed[col] = df_processed[col].fillna(-1)

    zero_fill_cols = [
        'cum_ecg_cnt', 'cum_stemi_cnt', 'cum_trop_cnt',
        'time_since_ed', 'time_since_last', 'trop_trend',
        'current_heart_rate', 'current_mean_bp', 'time_since_start_min'
    ]
    for col in zero_fill_cols:
        if col in df_processed.columns:
            df_processed[col] = df_processed[col].fillna(0)

    # ---------------------------------------------------------
    # C. 범주형 인코딩 (강력해진 부분!)
    # ---------------------------------------------------------
    # Gender (대소문자 처리)
    if 'gender' in df_processed.columns:
        df_processed['gender'] = df_processed['gender'].astype(str).str.upper().map({'M': 1, 'F': 0}).fillna(-1)

    # Arrival Transport (MIMIC 데이터 형식 반영)
    if 'arrival_transport' in df_processed.columns:
        # 소문자로 변환 후 공백 제거 ('WALK IN' -> 'walk in')
        transport_col = df_processed['arrival_transport'].astype(str).str.lower().str.strip()
        transport_map = {
            'ambulance': 2,
            'helicopter': 3,
            'walk in': 1,     # MIMIC 기본값
            'walk-in': 1,     # 변형 대응
            'walkin': 1,      # 변형 대응
            'unknown': 0,
            'other': 0
        }
        # 매핑되지 않는 값은 0으로 처리
        df_processed['arrival_transport'] = transport_col.map(transport_map).fillna(0)

    # ---------------------------------------------------------
    # D. 스케일링 (-1 유지)
    # ---------------------------------------------------------
    scaler = StandardScaler()
    scale_cols = ['last_trop', 'run_max_trop']
    for col in scale_cols:
        if col in df_processed.columns:
            valid_mask = df_processed[col].notna() & (df_processed[col] != -1) # 이미 -1인 것도 제외 안전장치
            if valid_mask.any():
                valid_data = df_processed.loc[valid_mask, [col]]
                df_processed.loc[valid_mask, col] = scaler.fit_transform(valid_data)
            df_processed[col] = df_processed[col].fillna(-1)

    # ---------------------------------------------------------
    # E. Target 생성
    # ---------------------------------------------------------
    if 'current_event_id' in df_processed.columns:
        df_processed['target_next_evt'] = df_processed.groupby('hadm_id')['current_event_id'].shift(-1)
        df_processed['target_next_evt'] = df_processed['target_next_evt'].fillna(EOS_TOKEN_ID).astype(int)

    if 'time_since_start_min' in df_processed.columns:
        next_time = df_processed.groupby('hadm_id')['time_since_start_min'].shift(-1)
        df_processed['target_time_to_next'] = next_time - df_processed['time_since_start_min']
        df_processed['target_time_to_next'] = df_processed['target_time_to_next'].fillna(0)

    # ---------------------------------------------------------
    # F. 마무리
    # ---------------------------------------------------------
    schema_cols = get_cohort_schema()
    for col in schema_cols:
        if col not in df_processed.columns:
            df_processed[col] = 0

    return df_processed[schema_cols]

# 실행
try:
    feature_matrix = preprocess_cohort_data(df_master)
    print("✅ 전처리 완료!")
    print("\n[수정 후 arrival_transport 값 분포 확인]")
    print(feature_matrix['arrival_transport'].value_counts()) # 이제 1, 2, 3이 보여야 함
except NameError:
    print("df_master를 로드해주세요.")



✅ 전처리 완료!

[수정 후 arrival_transport 값 분포 확인]
arrival_transport
0    14312
Name: count, dtype: int64


In [61]:
import pandas as pd
import os

# 1. 경로 설정 (제공해주신 경로)
base_path = '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스'
ed_base_path = os.path.join(base_path, 'mimic-iv-ed-2.2')
edstays_csv_path = os.path.join(ed_base_path, 'edstays.csv')

print(f"📂 파일 로드 시도: {edstays_csv_path}")

try:
    # 2. edstays 데이터 로드 (필요한 컬럼만 로드하여 메모리 절약)
    # hadm_id: 조인 키, arrival_transport: 필요한 정보
    edstays = pd.read_csv(edstays_csv_path, usecols=['hadm_id', 'arrival_transport'])
    print(f"✅ edstays 로드 완료! (행 개수: {len(edstays)})")

    # 3. feature_matrix 수정 작업 시작
    if 'feature_matrix' in globals():
        print("🔄 feature_matrix 업데이트를 시작합니다...")

        # (1) 기존의 잘못된(0으로 채워진) 컬럼 제거
        if 'arrival_transport' in feature_matrix.columns:
            feature_matrix = feature_matrix.drop(columns=['arrival_transport'])

        # (2) edstays 정보 병합 (Left Join)
        # 중복 제거 후 병합 (하나의 입원에 하나의 운송수단)
        edstays_dedup = edstays.drop_duplicates(subset='hadm_id')
        feature_matrix = feature_matrix.merge(edstays_dedup, on='hadm_id', how='left')

        # (3) 텍스트 매핑 (전처리)
        # 소문자 변환 및 공백 제거
        transport_series = feature_matrix['arrival_transport'].astype(str).str.lower().str.strip()

        # 매핑 사전 정의
        transport_map = {
            'ambulance': 2,
            'helicopter': 3,
            'walk in': 1,
            'walk-in': 1,
            'walkin': 1,
            'unknown': 0,
            'other': 0,
            'nan': 0,
            'none': 0
        }

        # 매핑 적용
        feature_matrix['arrival_transport'] = transport_series.map(transport_map).fillna(0).astype(int)

        # 4. 결과 확인
        print("\n✨ 수정 완료! 'arrival_transport' 값 분포:")
        print(feature_matrix['arrival_transport'].value_counts())

        # (선택사항) 원본 스키마 순서대로 컬럼 재정렬
        # feature_matrix = feature_matrix[get_cohort_schema()]

    else:
        print("❌ feature_matrix 변수가 메모리에 없습니다. 이전 단계 코드를 실행해주세요.")

except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다. 경로를 다시 확인해주세요:\n{edstays_csv_path}")
except Exception as e:
    print(f"❌ 오류 발생: {e}")

📂 파일 로드 시도: /content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/mimic-iv-ed-2.2/edstays.csv
✅ edstays 로드 완료! (행 개수: 425087)
🔄 feature_matrix 업데이트를 시작합니다...

✨ 수정 완료! 'arrival_transport' 값 분포:
arrival_transport
2    7574
1    3493
0    3093
3     152
Name: count, dtype: int64


In [62]:
feature_matrix

,subject_id,hadm_id,age,gender,race,cci_score,hfrs_score,door_to_ecg,door_to_trop,door_to_anti,...,time_since_start_min,current_event_id,prefix_events_str,current_heart_rate,current_mean_bp,target_mortality,target_next_evt,target_time_to_next,target_remain_los,arrival_transport
0,13647833,20006266,40,1,ASIAN - SOUTH EAST ASIAN,0,0.0,0,0,0,...,0,1,0,0,0,0,4,27504819.0,458714.683333,1
1,13647833,20006266,40,1,ASIAN - SOUTH EAST ASIAN,0,0.0,0,0,0,...,0,4,0,0,0,0,0,654.0,301.033333,1
2,13647833,20006266,40,1,ASIAN - SOUTH EAST ASIAN,0,0.0,0,0,0,...,0,0,0,0,0,0,2,626.0,290.133333,1
3,13647833,20006266,40,1,ASIAN - SOUTH EAST ASIAN,0,0.0,0,0,0,...,0,2,0,0,0,0,2,614.0,279.700000,1
4,13647833,20006266,40,1,ASIAN - SOUTH EAST ASIAN,0,0.0,0,0,0,...,0,2,0,0,0,0,7,16168.0,269.466667,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14307,16976054,29997425,70,1,WHITE,1,6.6,0,0,0,...,0,2,0,0,0,0,2,589.0,128.816667,1
14308,16976054,29997425,70,1,WHITE,1,6.6,0,0,0,...,0,2,0,0,0,0,2,120.0,119.000000,1
14309,16976054,29997425,70,1,WHITE,1,6.6,0,0,0,...,0,2,0,0,0,0,2,205.0,117.000000,1
14310,16976054,29997425,70,1,WHITE,1,6.6,0,0,0,...,0,2,0,0,0,0,7,6815.0,113.583333,1


In [63]:
import os

# 1. 최종 검증: 0 외에 1, 2, 3 등의 값이 잘 들어갔는지 확인
print("=== [최종 검증] feature_matrix 상태 확인 ===")

if 'arrival_transport' in feature_matrix.columns:
    counts = feature_matrix['arrival_transport'].value_counts()
    print("1. Arrival Transport 값 분포:")
    print(counts)

    # 0만 있다면 경고 (저장은 하되 알림)
    if len(counts) == 1 and 0 in counts:
        print("\n⚠️ 주의: 여전히 모든 값이 0입니다. edstays 병합이 제대로 안 되었을 수 있습니다.")
    else:
        print("\n✅ 확인 완료: 1(Walk-in), 2(Ambulance) 등의 값이 정상적으로 포함되어 있습니다.")
else:
    print("❌ 오류: 'arrival_transport' 컬럼이 존재하지 않습니다.")

# 2. CSV 파일로 저장
# 파일명 설정
file_name = 'cohort_ver122_feature_matrix.csv'

# 저장 경로 설정 (기존 base_path가 있다면 거기 저장, 없으면 현재 폴더)
if 'base_path' in globals():
    save_path = os.path.join(base_path, file_name)
else:
    save_path = file_name # 현재 작업 디렉토리

print(f"\n📂 저장 경로: {save_path}")

try:
    # 인덱스 제외하고 저장
    feature_matrix.to_csv(save_path, index=False)
    print(f"🎉 저장 성공! '{file_name}' 파일이 생성되었습니다.")
except Exception as e:
    print(f"❌ 저장 실패: {e}")

=== [최종 검증] feature_matrix 상태 확인 ===
1. Arrival Transport 값 분포:
arrival_transport
2    7574
1    3493
0    3093
3     152
Name: count, dtype: int64

✅ 확인 완료: 1(Walk-in), 2(Ambulance) 등의 값이 정상적으로 포함되어 있습니다.

📂 저장 경로: /content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/cohort_ver122_feature_matrix.csv
🎉 저장 성공! 'cohort_ver122_feature_matrix.csv' 파일이 생성되었습니다.


In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler

# =========================================================
# 1. 설정 및 데이터 로드 (edstays 병합)
# =========================================================
# 경로 설정 (사용자 환경에 맞게 수정됨)
base_path = '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스'
ed_base_path = os.path.join(base_path, 'mimic-iv-ed-2.2')
edstays_csv_path = os.path.join(ed_base_path, 'edstays.csv')

def get_cohort_schema():
    return [
        'subject_id', 'hadm_id', 'age', 'gender', 'race',
        'arrival_transport', 'cci_score', 'hfrs_score',
        'door_to_ecg', 'door_to_trop', 'door_to_anti', 'door_to_pci',
        'prefix_len', 'time_since_ed', 'time_since_last', 'is_night',
        'cum_ecg_cnt', 'cum_stemi_cnt', 'cum_trop_cnt',
        'stemi_flag', 'trop_pos_flag', 'last_trop', 'run_max_trop', 'trop_trend', 'pci_status',
        'pathway_stage', 'time_since_start_min', 'current_event_id', 'prefix_events_str',
        'current_heart_rate', 'current_mean_bp',
        'target_mortality', 'target_next_evt', 'target_time_to_next', 'target_remain_los'
    ]

def preprocess_cohort_data_final(df_input):
    print("🚀 전처리 프로세스 시작...")
    df = df_input.copy()

    # ---------------------------------------------------------
    # [Step 1] edstays 병합 (arrival_transport 복구)
    # ---------------------------------------------------------
    if 'arrival_transport' not in df.columns or df['arrival_transport'].eq(0).all():
        print("🔧 edstays 테이블 병합 시도...")
        try:
            # edstays 파일 로드 (hadm_id, arrival_transport만)
            if os.path.exists(edstays_csv_path):
                edstays = pd.read_csv(edstays_csv_path, usecols=['hadm_id', 'arrival_transport'])

                # 기존 잘못된 컬럼 제거
                if 'arrival_transport' in df.columns:
                    df = df.drop(columns=['arrival_transport'])

                # 중복 제거 후 병합
                edstays_dedup = edstays.drop_duplicates(subset='hadm_id')
                df = df.merge(edstays_dedup, on='hadm_id', how='left')
                print("✅ 병합 완료")
            else:
                print(f"⚠️ 경고: edstays 파일을 찾을 수 없습니다 ({edstays_csv_path})")
        except Exception as e:
            print(f"⚠️ 병합 중 오류 발생: {e}")

    # ---------------------------------------------------------
    # [Step 2] 시간 변수 '음수 -> 0' 보정 (요청사항 반영)
    # ---------------------------------------------------------
    # 1. time_since_ed (응급실 도착 후 경과 시간)
    if 'time_since_ed' in df.columns:
        # 음수는 0으로 처리 (Clipping)
        mask_neg = df['time_since_ed'] < 0
        if mask_neg.any():
            print(f"🛠️ time_since_ed 음수 값 {mask_neg.sum()}개를 0으로 보정합니다.")
            df.loc[mask_neg, 'time_since_ed'] = 0

    # 2. time_since_start_min (첫 이벤트 후 경과 시간)
    if 'time_since_start_min' in df.columns:
        # 음수는 0으로 처리 (첫 이벤트보다 시간이 앞설 수 없음)
        mask_neg_start = df['time_since_start_min'] < 0
        if mask_neg_start.any():
            print(f"🛠️ time_since_start_min 음수 값 {mask_neg_start.sum()}개를 0으로 보정합니다.")
            df.loc[mask_neg_start, 'time_since_start_min'] = 0

    # ---------------------------------------------------------
    # [Step 3] 데이터 정렬 (Forward Fill을 위해 필수)
    # ---------------------------------------------------------
    # hadm_id와 시간 순서대로 정렬
    if 'prefix_len' in df.columns:
        df = df.sort_values(by=['hadm_id', 'prefix_len'])
    elif 'time_since_start_min' in df.columns:
        df = df.sort_values(by=['hadm_id', 'time_since_start_min'])

    # ---------------------------------------------------------
    # [Step 4] Troponin Forward Fill (LOCF)
    # ---------------------------------------------------------
    # 이전에 검사한 값이 있다면 현재 결측치에 채워넣음
    ffill_cols = ['last_trop', 'run_max_trop']
    for col in ffill_cols:
        if col in df.columns:
            df[col] = df.groupby('hadm_id')[col].ffill()

    # ---------------------------------------------------------
    # [Step 5] 결측치(NaN) 처리
    # ---------------------------------------------------------
    # Delay 관련 (-1)
    delay_cols = ['door_to_ecg', 'door_to_trop', 'door_to_anti', 'door_to_pci']
    for col in delay_cols:
        if col in df.columns:
            df[col] = df[col].fillna(-1)

    # Count/Time 관련 (0)
    zero_fill_cols = [
        'cum_ecg_cnt', 'cum_stemi_cnt', 'cum_trop_cnt',
        'time_since_ed', 'time_since_last', 'trop_trend',
        'current_heart_rate', 'current_mean_bp', 'time_since_start_min'
    ]
    for col in zero_fill_cols:
        if col in df.columns:
            df[col] = df[col].fillna(0)

    # ---------------------------------------------------------
    # [Step 6] 범주형 매핑 (Mapping)
    # ---------------------------------------------------------
    # Gender
    if 'gender' in df.columns:
        df['gender'] = df['gender'].astype(str).str.upper().map({'M': 1, 'F': 0}).fillna(-1)

    # Arrival Transport (텍스트 -> 숫자)
    if 'arrival_transport' in df.columns:
        transport_series = df['arrival_transport'].astype(str).str.lower().str.strip()
        transport_map = {
            'ambulance': 2, 'helicopter': 3,
            'walk in': 1, 'walk-in': 1, 'walkin': 1,
            'unknown': 0, 'other': 0, 'nan': 0, 'none': 0
        }
        df['arrival_transport'] = transport_series.map(transport_map).fillna(0).astype(int)

    # ---------------------------------------------------------
    # [Step 7] 스케일링 (Troponin)
    # ---------------------------------------------------------
    scaler = StandardScaler()
    scale_cols = ['last_trop', 'run_max_trop']
    for col in scale_cols:
        if col in df.columns:
            # -1(미검사)이 아닌 유효 값만 스케일링
            valid_mask = df[col].notna() & (df[col] != -1)
            if valid_mask.any():
                df.loc[valid_mask, col] = scaler.fit_transform(df.loc[valid_mask, [col]])
            # 나머지는 -1 유지
            df[col] = df[col].fillna(-1)

    # ---------------------------------------------------------
    # [Step 8] Target 생성 (EOS=14)
    # ---------------------------------------------------------
    EOS_TOKEN_ID = 14

    # Next Event
    if 'current_event_id' in df.columns:
        df['target_next_evt'] = df.groupby('hadm_id')['current_event_id'].shift(-1)
        df['target_next_evt'] = df['target_next_evt'].fillna(EOS_TOKEN_ID).astype(int)

    # Time to Next
    if 'time_since_start_min' in df.columns:
        next_time = df.groupby('hadm_id')['time_since_start_min'].shift(-1)
        df['target_time_to_next'] = next_time - df['time_since_start_min']
        df['target_time_to_next'] = df['target_time_to_next'].fillna(0)

        # [추가 보정] target_time_to_next도 음수가 나오면 안 됨
        df.loc[df['target_time_to_next'] < 0, 'target_time_to_next'] = 0

    # ---------------------------------------------------------
    # [Step 9] 컬럼 정리 및 반환
    # ---------------------------------------------------------
    schema_cols = get_cohort_schema()
    for col in schema_cols:
        if col not in df.columns:
            df[col] = 0

    return df[schema_cols]

# =========================================================
# 실행 및 저장
# =========================================================
try:
    # 1. 전처리 실행 (feature_matrix 변수가 있다면 그것을, 없다면 df_master 사용)
    input_df = feature_matrix if 'feature_matrix' in globals() else df_master
    feature_matrix_final = preprocess_cohort_data_final(input_df)

    # 2. 결과 검증
    print("\n✅ 최종 데이터 검증:")
    print(f"   - 데이터 크기: {feature_matrix_final.shape}")

    # 시간 변수 음수 확인
    neg_ed = (feature_matrix_final['time_since_ed'] < 0).sum()
    neg_start = (feature_matrix_final['time_since_start_min'] < 0).sum()
    print(f"   - time_since_ed 음수 개수: {neg_ed} (0이어야 정상)")
    print(f"   - time_since_start_min 음수 개수: {neg_start} (0이어야 정상)")

    # 3. CSV 저장
    save_path = os.path.join(base_path, 'cohort_feature_matrix_final.csv')
    feature_matrix_final.to_csv(save_path, index=False)
    print(f"\n🎉 저장 완료: {save_path}")

except NameError:
    print("❌ 오류: 입력 데이터(feature_matrix 또는 df_master)가 메모리에 없습니다.")
except Exception as e:
    print(f"❌ 오류 발생: {e}")

❌ 오류: 입력 데이터(feature_matrix 또는 df_master)가 메모리에 없습니다.


In [4]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler

# =========================================================
# 1. 파일 경로 설정
# =========================================================
base_path = '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스'

# [입력] 작업할 파일명 (요청하신 ver122)
input_csv_name = 'cohort_ver122_feature_matrix.csv'
input_path = os.path.join(base_path, input_csv_name)

# [참조] edstays 파일 경로 (운송수단 복구용)
ed_base_path = os.path.join(base_path, 'mimic-iv-ed-2.2')
edstays_csv_path = os.path.join(ed_base_path, 'edstays.csv')

# [출력] 저장할 파일명
output_csv_name = 'cohort_ver122_feature_matrix_final.csv'
save_path = os.path.join(base_path, output_csv_name)

# =========================================================
# 2. 보정 및 전처리 함수
# =========================================================
def fix_and_finalize_data(df_input):
    print(f"🚀 '{input_csv_name}' 데이터 보정 시작...")
    df = df_input.copy()

    # ---------------------------------------------------------
    # [Step A] 시간 변수 '음수 -> 0' 강제 보정 (가장 중요!)
    # ---------------------------------------------------------
    # 1. time_since_ed (응급실 도착 후 경과 시간)
    if 'time_since_ed' in df.columns:
        mask_neg = df['time_since_ed'] < 0
        if mask_neg.any():
            print(f"🛠️ [Fix] time_since_ed 음수 값 {mask_neg.sum()}개를 0으로 보정합니다.")
            df.loc[mask_neg, 'time_since_ed'] = 0

    # 2. time_since_start_min (첫 이벤트 후 경과 시간)
    if 'time_since_start_min' in df.columns:
        mask_neg_start = df['time_since_start_min'] < 0
        if mask_neg_start.any():
            print(f"🛠️ [Fix] time_since_start_min 음수 값 {mask_neg_start.sum()}개를 0으로 보정합니다.")
            df.loc[mask_neg_start, 'time_since_start_min'] = 0

    # ---------------------------------------------------------
    # [Step B] edstays 병합 (arrival_transport가 0일 경우 복구)
    # ---------------------------------------------------------
    if 'arrival_transport' in df.columns:
        # 0이 아닌 값이 하나라도 있는지 확인
        if df['arrival_transport'].eq(0).all():
            print("🔧 [Fix] arrival_transport가 모두 0입니다. edstays 병합을 시도합니다.")
            try:
                if os.path.exists(edstays_csv_path):
                    edstays = pd.read_csv(edstays_csv_path, usecols=['hadm_id', 'arrival_transport'])
                    df = df.drop(columns=['arrival_transport']) # 기존 컬럼 제거
                    edstays_dedup = edstays.drop_duplicates(subset='hadm_id')
                    df = df.merge(edstays_dedup, on='hadm_id', how='left')
                    print("✅ 병합 완료")
                else:
                    print("⚠️ edstays 파일이 없어 병합을 건너뜁니다.")
            except Exception as e:
                print(f"⚠️ 병합 중 오류: {e}")

    # ---------------------------------------------------------
    # [Step C] 범주형 매핑 (대소문자/공백 처리)
    # ---------------------------------------------------------
    if 'arrival_transport' in df.columns and df['arrival_transport'].dtype == 'O':
         # 텍스트인 경우에만 매핑 수행 (이미 숫자면 건너뜀)
        transport_series = df['arrival_transport'].astype(str).str.lower().str.strip()
        transport_map = {
            'ambulance': 2, 'helicopter': 3,
            'walk in': 1, 'walk-in': 1, 'walkin': 1,
            'unknown': 0, 'other': 0, 'nan': 0, 'none': 0, '0': 0
        }
        df['arrival_transport'] = transport_series.map(transport_map).fillna(0).astype(int)

    # ---------------------------------------------------------
    # [Step D] 데이터 정렬 (중요)
    # ---------------------------------------------------------
    if 'hadm_id' in df.columns:
        sort_cols = ['hadm_id']
        if 'prefix_len' in df.columns:
            sort_cols.append('prefix_len')
        elif 'time_since_start_min' in df.columns:
            sort_cols.append('time_since_start_min')

        df = df.sort_values(by=sort_cols)

    # ---------------------------------------------------------
    # [Step E] Target 생성 및 보정
    # ---------------------------------------------------------
    EOS_TOKEN_ID = 14

    # Next Event
    if 'current_event_id' in df.columns:
        df['target_next_evt'] = df.groupby('hadm_id')['current_event_id'].shift(-1)
        df['target_next_evt'] = df['target_next_evt'].fillna(EOS_TOKEN_ID).astype(int)

    # Time to Next (음수 방지 포함)
    if 'time_since_start_min' in df.columns:
        next_time = df.groupby('hadm_id')['time_since_start_min'].shift(-1)
        df['target_time_to_next'] = next_time - df['time_since_start_min']
        df['target_time_to_next'] = df['target_time_to_next'].fillna(0)

        # [Fix] Target 시간 음수 보정
        mask_neg_target = df['target_time_to_next'] < 0
        if mask_neg_target.any():
            print(f"🛠️ [Fix] target_time_to_next 음수 값 {mask_neg_target.sum()}개를 0으로 보정합니다.")
            df.loc[mask_neg_target, 'target_time_to_next'] = 0

    return df

# =========================================================
# 3. 실행 로직
# =========================================================
try:
    print(f"📂 파일 로드 중: {input_path}")

    if os.path.exists(input_path):
        # 1. CSV 로드
        df_ver122 = pd.read_csv(input_path)
        print(f"✅ 로드 성공! 데이터 크기: {df_ver122.shape}")

        # 2. 보정 함수 실행
        df_final = fix_and_finalize_data(df_ver122)

        # 3. 결과 확인 (음수 여부)
        neg_check_1 = (df_final['time_since_ed'] < 0).sum() if 'time_since_ed' in df_final.columns else 0
        neg_check_2 = (df_final['time_since_start_min'] < 0).sum() if 'time_since_start_min' in df_final.columns else 0

        print("\n🔎 최종 검증:")
        print(f"   - time_since_ed 음수 개수: {neg_check_1}")
        print(f"   - time_since_start_min 음수 개수: {neg_check_2}")
        if 'arrival_transport' in df_final.columns:
             print(f"   - arrival_transport 값 분포:\n{df_final['arrival_transport'].value_counts().head()}")

        # 4. 저장
        df_final.to_csv(save_path, index=False)
        print(f"\n🎉 저장 완료! 경로: {save_path}")

    else:
        print(f"❌ 파일을 찾을 수 없습니다: {input_path}")
        print("   파일명을 다시 확인하거나 구글 드라이브 경로를 확인해주세요.")

except Exception as e:
    print(f"❌ 오류 발생: {e}")

📂 파일 로드 중: /content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/cohort_ver122_feature_matrix.csv
✅ 로드 성공! 데이터 크기: (14312, 35)
🚀 'cohort_ver122_feature_matrix.csv' 데이터 보정 시작...
🛠️ [Fix] time_since_ed 음수 값 1530개를 0으로 보정합니다.

🔎 최종 검증:
   - time_since_ed 음수 개수: 0
   - time_since_start_min 음수 개수: 0
   - arrival_transport 값 분포:
arrival_transport
2    7574
1    3493
0    3093
3     152
Name: count, dtype: int64

🎉 저장 완료! 경로: /content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/cohort_ver122_feature_matrix_final.csv


In [7]:
import pandas as pd
import numpy as np
import os

# =========================================================
# 1. 파일 경로 설정
# =========================================================
base_path = '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스'

# 입력 파일: 문제가 있던 ver122 파일
input_csv_name = 'cohort_ver122_feature_matrix.csv'
input_path = os.path.join(base_path, input_csv_name)

# 출력 파일: 수리가 완료된 파일
output_csv_name = 'cohort_ver123_feature_matrix_repaired.csv'
save_path = os.path.join(base_path, output_csv_name)

# =========================================================
# 2. 핵심 복구 로직 함수
# =========================================================
def repair_time_features(df_input):
    df = df_input.copy()
    print(f"🚀 데이터 로드 완료: {df.shape}")

    # [중요] 1. 데이터 정렬 (계산을 위해 필수)
    # 입원ID(hadm_id)별로 묶고, 순서(prefix_len)대로 줄 세우기
    if 'hadm_id' in df.columns and 'prefix_len' in df.columns:
        df = df.sort_values(by=['hadm_id', 'prefix_len'])
        print("✅ 정렬 완료 (hadm_id -> prefix_len)")

    # -----------------------------------------------------
    # [Step A] 기반 데이터(time_since_ed) 음수 보정
    # -----------------------------------------------------
    # 재료가 되는 시간이 음수면 결과도 이상해지므로 먼저 0으로 맞춥니다.
    if 'time_since_ed' in df.columns:
        mask_neg = df['time_since_ed'] < 0
        if mask_neg.any():
            print(f"🛠️ time_since_ed 음수 값 {mask_neg.sum()}개를 0으로 보정합니다.")
            df.loc[mask_neg, 'time_since_ed'] = 0

    # -----------------------------------------------------
    # [Step B] time_since_start_min 재계산 (핵심!)
    # -----------------------------------------------------
    # 공식: (현재 시간) - (그 환자의 첫 번째 이벤트 시간)
    if 'time_since_ed' in df.columns:
        print("🔄 time_since_start_min 재계산 중...")

        # 1. 각 hadm_id 그룹별 '첫 번째 time_since_ed' 값을 찾아서 모든 행에 붙여줌
        first_times = df.groupby('hadm_id')['time_since_ed'].transform('first')

        # 2. 현재 시간에서 첫 시간을 빼서 '경과 시간'을 구함
        df['time_since_start_min'] = df['time_since_ed'] - first_times

        # 3. 혹시나 계산 결과가 음수(데이터 오류)라면 0으로 처리
        df.loc[df['time_since_start_min'] < 0, 'time_since_start_min'] = 0

    # -----------------------------------------------------
    # [Step C] Target 변수(Next Time) 동기화
    # -----------------------------------------------------
    # 기준 시간이 바뀌었으니, "다음 이벤트까지 남은 시간"도 다시 계산해야 정확합니다.
    if 'time_since_start_min' in df.columns:
        print("🎯 Target(target_time_to_next) 업데이트 중...")
        # 다음 행의 시간 - 현재 행의 시간
        next_time = df.groupby('hadm_id')['time_since_start_min'].shift(-1)
        df['target_time_to_next'] = next_time - df['time_since_start_min']

        # 마지막 이벤트(NaN)와 음수 값 처리
        df['target_time_to_next'] = df['target_time_to_next'].fillna(0)
        df.loc[df['target_time_to_next'] < 0, 'target_time_to_next'] = 0

    return df

# =========================================================
# 3. 실행 및 결과 확인
# =========================================================
try:
    if os.path.exists(input_path):
        # 로드
        df_ver122 = pd.read_csv(input_path)

        # 복구 실행
        df_repaired = repair_time_features(df_ver122)

        # 검증 출력
        print("\n🔎 [검증] hadm_id 별 시간 변화 확인 (0부터 시작해서 증가해야 함):")
        cols_to_check = ['hadm_id', 'prefix_len', 'time_since_ed', 'time_since_start_min']
        print(df_repaired[cols_to_check].head(10).to_markdown(index=False))

        # 저장
        df_repaired.to_csv(save_path, index=False)
        print(f"\n🎉 저장 완료! 경로: {save_path}")

    else:
        print(f"❌ 파일을 찾을 수 없습니다: {input_path}")

except Exception as e:
    print(f"❌ 오류 발생: {e}")

df_repaired

🚀 데이터 로드 완료: (14312, 35)
✅ 정렬 완료 (hadm_id -> prefix_len)
🛠️ time_since_ed 음수 값 1530개를 0으로 보정합니다.
🔄 time_since_start_min 재계산 중...
🎯 Target(target_time_to_next) 업데이트 중...

🔎 [검증] hadm_id 별 시간 변화 확인 (0부터 시작해서 증가해야 함):
|     hadm_id |   prefix_len |   time_since_ed |   time_since_start_min |
|------------:|-------------:|----------------:|-----------------------:|
| 2.00063e+07 |            1 |               0 |                      0 |
| 2.00063e+07 |            2 |               0 |                      0 |
| 2.00063e+07 |            3 |               0 |                      0 |
| 2.00063e+07 |            4 |             626 |                    626 |
| 2.00063e+07 |            5 |            1240 |                   1240 |
| 2.00063e+07 |            6 |           17408 |                  17408 |
| 2.00143e+07 |            1 |               0 |                      0 |
| 2.00143e+07 |            2 |               5 |                      5 |
| 2.00143e+07 |            3 |               

,subject_id,hadm_id,age,gender,race,cci_score,hfrs_score,door_to_ecg,door_to_trop,door_to_anti,...,time_since_start_min,current_event_id,prefix_events_str,current_heart_rate,current_mean_bp,target_mortality,target_next_evt,target_time_to_next,target_remain_los,arrival_transport
0,13647833,20006266,40,1,ASIAN - SOUTH EAST ASIAN,0,0.0,0,0,0,...,0.0,1,0,0,0,0,4,0.0,458714.683333,1
1,13647833,20006266,40,1,ASIAN - SOUTH EAST ASIAN,0,0.0,0,0,0,...,0.0,4,0,0,0,0,0,0.0,301.033333,1
2,13647833,20006266,40,1,ASIAN - SOUTH EAST ASIAN,0,0.0,0,0,0,...,0.0,0,0,0,0,0,2,626.0,290.133333,1
3,13647833,20006266,40,1,ASIAN - SOUTH EAST ASIAN,0,0.0,0,0,0,...,626.0,2,0,0,0,0,2,614.0,279.700000,1
4,13647833,20006266,40,1,ASIAN - SOUTH EAST ASIAN,0,0.0,0,0,0,...,1240.0,2,0,0,0,0,7,16168.0,269.466667,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14307,16976054,29997425,70,1,WHITE,1,6.6,0,0,0,...,3759.0,2,0,0,0,0,2,589.0,128.816667,1
14308,16976054,29997425,70,1,WHITE,1,6.6,0,0,0,...,4348.0,2,0,0,0,0,2,120.0,119.000000,1
14309,16976054,29997425,70,1,WHITE,1,6.6,0,0,0,...,4468.0,2,0,0,0,0,2,205.0,117.000000,1
14310,16976054,29997425,70,1,WHITE,1,6.6,0,0,0,...,4673.0,2,0,0,0,0,7,6815.0,113.583333,1
